In [1]:
import csv
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import radioactivedecay as rd
from pypost.codes.melcor import MELCOR

# =======================
# INPUT FROM MELCOR (same variable structure)
# =======================

# --- Adjust here the file and the CVHs you want to use ---
filename = "MC_Step.ptf"
cv_list = [110, 160, 200, 305, 1, 600, 201]

# --- Adjust here the CFVALU range that previously came from JSON ---
start_number = 3000     # <-- set your real start
end_number   = 3062     # <-- set your real end

# Open MELCOR file
index = MELCOR.openPlotFile(filename)

# Take time values from the first reference variable (as requested)
primer_cv = cv_list[0]
var_ref = f"CVH-VOLLIQ_{primer_cv}"
data_ref = np.array(MELCOR.getData(index, var_ref))

t_ini_raw = float(data_ref[0, 0])
t_fin_raw = float(data_ref[-1, 0])

# +1 s and truncated (remove decimals)
start_time = int(t_ini_raw + 1)
end_time   = int(t_fin_raw + 1)

# =======================
# Calculation of V_values (keys 1..N) and index->CVH map
# =======================
V_values = {}                 # dict[int -> float] in cm³; keys 1..N
cv_index_map = {}             # dict[int -> int]  index->CVH (for info only)

for idx, cv in enumerate(cv_list, start=1):   # idx = 1..N
    cv_index_map[idx] = int(cv)

    var_liq = f"CVH-VOLLIQ_{cv}"
    var_vap = f"CVH-VOLVAP_{cv}"

    data_liq = np.array(MELCOR.getData(index, var_liq))
    data_vap = np.array(MELCOR.getData(index, var_vap))

    t_liq = data_liq[:, 0]
    v_liq = data_liq[:, 1]
    t_vap = data_vap[:, 0]
    v_vap = data_vap[:, 1]

    # Align time arrays if needed
    if (data_liq.shape[0] != data_vap.shape[0]) or (not np.allclose(t_liq, t_vap)):
        v_vap = np.interp(t_liq, t_vap, v_vap)

    total_m3 = v_liq + v_vap
    avg_m3   = float(np.mean(total_m3))
    avg_cm3  = avg_m3 * 1_000_000.0  # m³ -> cm³

    # Key = sequential index (1..N), not the CVH number
    V_values[idx] = avg_cm3

# Build CFVALU and batches (identical to your structure)
mass_data_lines = [f"CFVALU_{i}" for i in range(start_number, end_number + 1)]
batches = [mass_data_lines[i:i + 9] for i in range(0, len(mass_data_lines), 9)]
num_batches = len(batches)

# Messages (same organization and variables; only data source changes)
print(f"\n[INFO] Parameters loaded from MELCOR (no JSON)")
print(f" - Control functions from {start_number} to {end_number}")
print(f" - Simulation time: {start_time}s to {end_time}s")
print(f" - Number of control volumes (batches): {num_batches}")
print(f" - Volumes (keys 1..N, cm³): {V_values}")

# (Optional) Show index -> CVH mapping
print(f" - Index->CVH map: {cv_index_map}")






[INFO] Parameters loaded from MELCOR (no JSON)
 - Control functions from 3000 to 3062
 - Simulation time: 1s to 107288s
 - Number of control volumes (batches): 7
 - Volumes (keys 1..N, cm³): {1: 144162662.5198637, 2: 123851651.02197042, 3: 565922975.1899465, 4: 6471758280.302349, 5: 79999998.11147389, 6: 1e+16, 7: 6509367032.222646}
 - Index->CVH map: {1: 110, 2: 160, 3: 200, 4: 305, 5: 1, 6: 600, 7: 201}


In [2]:
# ================================
# OUTPUTS Y RADIONUCLIDE CLASSES
# ================================
output_directory_1 = "proportions"
recalculate_proportions = False

if not os.path.exists(output_directory_1):
    os.makedirs(output_directory_1)
    recalculate_proportions = True
else:
    expected_files = [f"class_{i}_proportions.csv" for i in range(1, 10)]
    existing_files = os.listdir(output_directory_1)
    if not all(f in existing_files for f in expected_files):
        recalculate_proportions = True

with open("class_inventories.json", "r") as f:
    class_inventories = json.load(f)

# Convertir claves a enteros (opcional pero recomendable)
class_inventories = {int(k): v for k, v in class_inventories.items()}        

time_steps = np.arange(start_time, end_time, 1)
all_class_proportions = {}


In [3]:
import csv
import pandas as pd


# ==========================================
# CALCULATION OR LOADING OF PROPORTIONS
# ==========================================
output_directory_1 = "proportions"
os.makedirs(output_directory_1, exist_ok=True)
expected_files = [f"class_{i}_proportions.csv" for i in range(1, 10)]
recalc = not all(f in os.listdir(output_directory_1) for f in expected_files)

all_class_proportions = {}

if recalc:
    for class_num, inv_props in class_inventories.items():
        print(f"Calculating proportions for class {class_num}...")
        inv = rd.Inventory({}, "kg")
        # initialize 1 kg total distributed according to inventory proportions
        inv += rd.Inventory({iso: 1.0 * prop for iso, prop in inv_props.items()}, "kg")
        proportions = {iso: np.zeros(len(time_steps)) for iso in inv_props}
        last_t = time_steps[0]
        for i, t in enumerate(time_steps):
            inv = inv.decay(t - last_t, 's') if i > 0 else inv
            last_t = t
            masses = inv.masses("kg")
            total = sum(masses.values())
            for iso in proportions:
                proportions[iso][i] = max(masses.get(iso, 0), 0) / total if total > 0 else 0

        all_class_proportions[class_num] = proportions

        # save CSV
        csv_path = os.path.join(output_directory_1, f"class_{class_num}_proportions.csv")
        with open(csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Time (s)"] + list(proportions.keys()))
            for i, t in enumerate(time_steps):
                writer.writerow([t] + [proportions[iso][i] for iso in proportions])

        # plot results
        plt.figure(figsize=(8, 5))
        for iso, arr in proportions.items():
            plt.plot(time_steps, arr, label=iso)
        plt.legend(); plt.grid(True)
        plt.savefig(os.path.join(output_directory_1, f"class_{class_num}_proportions.png"))
        plt.close()

else:
    for class_num in range(1, 10):
        csv_path = os.path.join(output_directory_1, f"class_{class_num}_proportions.csv")
        df = pd.read_csv(csv_path)
        isotopes = df.columns.tolist()[1:]
        all_class_proportions[class_num] = {
            iso: df[iso].values for iso in isotopes
        }
    print("Proportions successfully loaded.")



Proportions successfully loaded.


In [4]:
import os
import csv
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from pypost.codes.melcor import MELCOR

# ==========================================
# MODULE: CALCULATION OR LOADING OF MASSES (all batches)
# ==========================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Masses ===")

    # 1) Output directory, create if necessary
    output_directory = f"output_batch_{batch_num}"
    os.makedirs(output_directory, exist_ok=True)

    # 2) Do the mass CSV files already exist?
    expected = [f"class_{i}_masses.csv" for i in range(1, 10)]
    recalc = not all(fn in os.listdir(output_directory) for fn in expected)

    all_class_masses = {}

    if recalc:
        print("[INFO] Calculating masses from MELCOR…")
        batch = batches[batch_num - 1]               # the 9 variables of this batch

        # Loop through each of the 9 classes
        for class_num, var in enumerate(batch, start=1):
            print(f"  Class {class_num}: variable '{var}'")
            # 2.1) Read from MELCOR
            idx = MELCOR.openPlotFile("MC_Step.ptf")
            data = np.array(MELCOR.getData(idx, var))
            t, M = data[:, 0], data[:, 1]

            # 2.2) Filter and make uniform
            mask = (t >= start_time) & (t <= end_time)
            uni_t = np.arange(start_time, end_time, 1)
            uni_M = interp1d(t[mask], M[mask],
                             kind='previous',
                             fill_value="extrapolate")(uni_t)

            # 2.3) Distribute according to proportions
            props = all_class_proportions[class_num]
            masses = {iso: props[iso] * uni_M for iso in props}
            all_class_masses[class_num] = masses

            # 2.4) Save to CSV
            csv_path = os.path.join(output_directory,
                                    f"class_{class_num}_masses.csv")
            with open(csv_path, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + list(masses.keys()))
                for i, t0 in enumerate(uni_t):
                    w.writerow([t0] + [masses[iso][i] for iso in masses])

            # 2.5) Plot results
            plt.figure(figsize=(8, 5))
            for iso, arr in masses.items():
                plt.plot(uni_t, arr, label=iso)
            plt.xlabel("Time (s)")
            plt.ylabel("Mass (kg)")
            plt.title(f"Batch {batch_num} – Class {class_num} Masses")
            plt.legend(); plt.grid(True)
            plt.savefig(os.path.join(output_directory,
                                     f"class_{class_num}_masses.png"))
            plt.close()

        print(f"[Batch {batch_num}] Masses calculated and saved.")

    else:
        print("[INFO] Loading masses from CSV…")
        for class_num in range(1, 10):
            csv_path = os.path.join(output_directory,
                                    f"class_{class_num}_masses.csv")
            df = pd.read_csv(csv_path)
            isotopes = df.columns.tolist()[1:]
            all_class_masses[class_num] = {
                iso: df[iso].values for iso in isotopes
            }
        print(f"[Batch {batch_num}] Masses successfully reloaded.")

    # Now `all_class_masses` is ready for this batch_num
    # Quick check:
    print(f"  Class 1 – first mass value for '{list(all_class_masses[1].keys())[0]}' =",
          all_class_masses[1][list(all_class_masses[1].keys())[0]][0])





=== Batch 1 — Masses ===
[INFO] Loading masses from CSV…
[Batch 1] Masses successfully reloaded.
  Class 1 – first mass value for 'Kr-84' = nan

=== Batch 2 — Masses ===
[INFO] Loading masses from CSV…
[Batch 2] Masses successfully reloaded.
  Class 1 – first mass value for 'Kr-84' = nan

=== Batch 3 — Masses ===
[INFO] Loading masses from CSV…
[Batch 3] Masses successfully reloaded.
  Class 1 – first mass value for 'Kr-84' = nan

=== Batch 4 — Masses ===
[INFO] Loading masses from CSV…
[Batch 4] Masses successfully reloaded.
  Class 1 – first mass value for 'Kr-84' = nan

=== Batch 5 — Masses ===
[INFO] Loading masses from CSV…
[Batch 5] Masses successfully reloaded.
  Class 1 – first mass value for 'Kr-84' = nan

=== Batch 6 — Masses ===
[INFO] Loading masses from CSV…
[Batch 6] Masses successfully reloaded.
  Class 1 – first mass value for 'Kr-84' = nan

=== Batch 7 — Masses ===
[INFO] Loading masses from CSV…
[Batch 7] Masses successfully reloaded.
  Class 1 – first mass value for

In [5]:

import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import radioactivedecay as rd

# ==========================================
# MODULE: CALCULATION OR LOADING OF ACTIVITIES (all batches)
# ==========================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Activities ===")

    # 1) Output directory and subfolder for plots
    output_directory = f"output_batch_{batch_num}"
    dose_plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs(dose_plots_dir, exist_ok=True)

    # 2) Reload MASSES from CSV
    all_class_masses = {}
    for class_num in range(1, 10):
        mass_csv = os.path.join(output_directory, f"class_{class_num}_masses.csv")
        df_mass = pd.read_csv(mass_csv)
        isotopes = df_mass.columns.tolist()[1:]
        all_class_masses[class_num] = {iso: df_mass[iso].values for iso in isotopes}

    # 3) Check for existing ACTIVITY CSV files
    expected = [f"class_{i}_activities.csv" for i in range(1, 10)]
    recalc_act = not all(fn in os.listdir(output_directory) for fn in expected)

    all_class_activities = {}

    if recalc_act:
        print("[INFO] Calculating activities from masses…")
        for class_num, isotope_masses in all_class_masses.items():
            print(f"  Class {class_num}")
            # Prepare the activity dictionary (Ci)
            acts = {iso: np.zeros(len(time_steps)) for iso in isotope_masses}

            for idx, t in enumerate(time_steps):
                inv = rd.Inventory({}, "kg")
                for iso, mass_arr in isotope_masses.items():
                    m = mass_arr[idx]
                    if m > 0:
                        inv += rd.Inventory({iso: m}, "kg")
                bq_dict = inv.activities("Bq")
                for iso, bq in bq_dict.items():
                    acts[iso][idx] = bq / 3.7e10  # Convert Bq → Ci

            all_class_activities[class_num] = acts

            # Save CSV
            csv_path = os.path.join(output_directory, f"class_{class_num}_activities.csv")
            with open(csv_path, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + list(acts.keys()))
                for i, t in enumerate(time_steps):
                    w.writerow([t] + [acts[iso][i] for iso in acts])

            # Plot
            plt.figure(figsize=(8, 5))
            for iso, arr in acts.items():
                plt.plot(time_steps, arr, label=iso)
            plt.xlabel("Time (s)")
            plt.ylabel("Activity (Ci)")
            plt.title(f"Batch {batch_num} – Class {class_num} Activities")
            plt.legend(); plt.grid(True)
            plt.savefig(os.path.join(output_directory, f"class_{class_num}_activities.png"))
            plt.close()

        print(f"[Batch {batch_num}] Activities calculated and saved.")

    else:
        print("[INFO] Loading activities from CSV…")
        for class_num in range(1, 10):
            csv_path = os.path.join(output_directory, f"class_{class_num}_activities.csv")
            df_act = pd.read_csv(csv_path)
            isotopes = df_act.columns.tolist()[1:]
            all_class_activities[class_num] = {
                iso: df_act[iso].values for iso in isotopes
            }
        print(f"[Batch {batch_num}] Activities successfully reloaded.")

    # Now you can use all_class_activities[batch_num]...






=== Batch 1 — Activities ===
[INFO] Loading activities from CSV…
[Batch 1] Activities successfully reloaded.

=== Batch 2 — Activities ===
[INFO] Loading activities from CSV…
[Batch 2] Activities successfully reloaded.

=== Batch 3 — Activities ===
[INFO] Loading activities from CSV…
[Batch 3] Activities successfully reloaded.

=== Batch 4 — Activities ===
[INFO] Loading activities from CSV…
[Batch 4] Activities successfully reloaded.

=== Batch 5 — Activities ===
[INFO] Loading activities from CSV…
[Batch 5] Activities successfully reloaded.

=== Batch 6 — Activities ===
[INFO] Loading activities from CSV…
[Batch 6] Activities successfully reloaded.

=== Batch 7 — Activities ===
[INFO] Loading activities from CSV…
[Batch 7] Activities successfully reloaded.


In [6]:
import json

with open("energies_data.json", "r") as f:
    data = json.load(f)

gamma_energies = data["gamma_energies"]
beta_emitter_energies = data["beta_emitter_energies"]


In [7]:
import os
import csv
import numpy as np
import pandas as pd

# ==========================================
# MODULE: ADD BREMSSTRAHLUNG CONTRIBUTION
# ==========================================
# Assumes gamma_energies and beta_emitter_energies are already defined elsewhere

Z_air = 7.3  # effective atomic number of air

for iso, E_beta in beta_emitter_energies.items():
    # Add bremsstrahlung energy (in MeV)
    E_brem = 1.33e-4 * Z_air * E_beta**2
    gamma_energies[iso] = gamma_energies.get(iso, 0) + E_brem

print("[INFO] Bremsstrahlung energy added to gamma_energies for all isotopes.")

# ==========================================
# SKIPPING AVERAGE GAMMA ENERGY CALCULATION
# ==========================================
# This block previously calculated the average gamma energy per class and per time step.
# It has been intentionally removed since only the bremsstrahlung correction is needed.




[INFO] Bremsstrahlung energy added to gamma_energies for all isotopes.


In [8]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import json

# ==================================
# LOAD GAMMA ENERGIES (with bremsstrahlung already added)
# ==================================
with open("energies_data.json", "r") as f:
    energies_data = json.load(f)

gamma_energies = energies_data["gamma_energies"]

# ==================================
# CONSTANTS FOR DOSE RATE CALCULATION
# ==================================

# These are updated dynamically per batch in the next cell
density = 0.001  # g/cm³

# ==================================
# MASS ATTENUATION AND ABSORPTION FUNCTIONS
# ==================================

def mass_att(E, a1=-0.037274, b1=0.101714, c1=-0.274123):
    """Mass attenuation coefficient as a function of photon energy E (MeV)."""
    if E <= 0:
        return 0
    return a1 + b1 * E**c1

def mass_ab(E, a2=0, b2=-5.2588e-4, c2=-5.2077e-3, d=2.8172e-2, e=-1.7809e-2):
    """Mass absorption coefficient as a function of photon energy E (MeV)."""
    if E <= 0:
        return 0
    return a2 + b2 * E + c2 * (math.log(E))**2 + d * math.sqrt(E) + e * math.log(E)

def calc_mu_m(E):
    """Compute μm depending on photon energy E (keV)."""
    if 0 <= E <= 200:
        B0, B1, B2, B3 = 0.29839, -0.00269, 1.67948E-5, -3.75963E-8
        mu_m = B3 * E**3 + B2 * E**2 + B1 * E + B0
    elif E > 200:
        B0, B1, B2 = 0.13556, -9.10106E-5, 2.39846E-8
        mu_m = B2 * E**2 + B1 * E + B0
    else:
        return 0
    return mu_m

# ==================================
# CIRCULAR BASE METHOD FOR GAMMA FLUX
# ==================================

def gamma_flux_circular_base(activity, R, h, mass_att_func, E):
    """Computes gamma flux using the circular base method."""
    if E <= 0 or activity <= 0:
        return 0.0
    mu = mass_att_func(E)
    if mu <= 0:
        return 0.0

    r = np.linspace(0, R, 10)
    theta = np.linspace(0, np.arctan(R / h), 10)
    R_grid, Theta_grid = np.meshgrid(r, theta)
    path_length = np.sqrt(R_grid**2 + h**2)
    b1 = mu * path_length
    E1_term = sp.exp1(b1)
    E1_sec_term = sp.exp1(b1 / np.cos(Theta_grid))
    integrand = (((3.7e10) * activity) / (2 * np.pi * R**2)) * (E1_term - E1_sec_term)
    result = np.trapz(np.trapz(integrand * R_grid, r, axis=0), theta)
    return result

# ==================================
# PLOTTING FUNCTIONS FOR DOSE RATES
# ==================================

def plot_dose_rates(dose_rates, title, filename, out_dir):
    """Plot dose rate evolution for multiple isotopes."""
    plt.figure(figsize=(10, 6))
    for iso, dr in dose_rates.items():
        plt.plot(time_steps, dr, label=f"{iso} Dose Rate (rads/h)")
    plt.xlabel("Time (s)")
    plt.ylabel("Dose Rate (rads/h)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(out_dir, filename))
    plt.close()

def plot_accumulated_dose(total_dose, title, filename, out_dir):
    """Plot total accumulated dose as a function of time."""
    plt.figure(figsize=(10, 6))
    plt.plot(time_steps, total_dose, label="Total Accumulated Dose (rads)")
    plt.xlabel("Time (s)")
    plt.ylabel("Accumulated Dose (rads)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(out_dir, filename))
    plt.close()


In [9]:
import os
import csv
import numpy as np
import pandas as pd

# ===============================================
# MODULE: CALCULATION OR LOADING OF DOSE RATE (Sphere Hypothesis)
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Dose Rates (Sphere) ===")

    # 1) Output directories and control volume
    output_directory = f"output_batch_{batch_num}"
    plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(plots_dir, exist_ok=True)
    V = V_values[batch_num]

    # 2) Reload ACTIVITIES (gamma energies come from JSON)
    all_class_activities = {}
    for class_num in range(1, 10):
        act_csv = os.path.join(output_directory, f"class_{class_num}_activities.csv")
        df_act = pd.read_csv(act_csv)
        isotopes = df_act.columns.tolist()[1:]
        all_class_activities[class_num] = {iso: df_act[iso].values for iso in isotopes}

    # 3) Geometric parameters
    R_sphere = ((3 * V) / (4 * np.pi)) ** (1 / 3)
    density = 0.001  # g/cm³

    # 4) Check for per-class CSV files
    class_csvs = [f"dose_rate_sphere_class_{i}.csv" for i in range(1, 10)]
    have_class = all(fn in os.listdir(plots_dir) for fn in class_csvs)

    # 5) Check for total accumulated CSV
    total_csv = "total_accumulated_dose_sphere_all_classes.csv"
    have_total = total_csv in os.listdir(plots_dir)

    total_accumulated = np.zeros(len(time_steps))

    # ===============================================
    # CALCULATE IF MISSING
    # ===============================================
    if not have_class:
        print("[INFO] Missing per-class dose rate CSVs → recalculating everything…")

        for class_num, isotopes in class_inventories.items():
            acts = all_class_activities[class_num]
            dose_rates = {iso: np.zeros(len(time_steps)) for iso in isotopes}
            total_per_time = np.zeros(len(time_steps))

            for i, t in enumerate(time_steps):
                step_sum = 0.0
                for iso in isotopes:
                    E = gamma_energies.get(iso, 0.0)
                    if E <= 0:
                        continue
                    A = acts.get(iso, np.zeros(len(time_steps)))[i]
                    if A <= 0:
                        continue

                    mu = mass_att(E)
                    sigma = max(1e-4, mass_ab(E))
                    if mu == 0:
                        continue

                    flux = (3.7e10) * (A / (V * density * mu)) * (1 - np.exp(-density * R_sphere * mu))
                    dr = 5.77e-5 * flux * E * sigma
                    dose_rates[iso][i] = dr
                    step_sum += dr

                total_per_time[i] = step_sum / 3600.0  # rads/h

            # Accumulate dose
            acc = 0.0
            for i in range(len(time_steps)):
                acc += total_per_time[i]
                total_accumulated[i] += acc

            # Save per-class CSV
            out_csv = os.path.join(plots_dir, f"dose_rate_sphere_class_{class_num}.csv")
            with open(out_csv, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["Time (s)"] + list(dose_rates.keys()))
                for j, tt in enumerate(time_steps):
                    writer.writerow([tt] + [dose_rates[iso][j] for iso in dose_rates])

            # Plot per-class dose rates
            plot_dose_rates(
                dose_rates,
                f"Dose Rates (Sphere) - Class {class_num}",
                f"dose_rates_sphere_class_{class_num}.png",
                out_dir=plots_dir
            )

    # ===============================================
    # IF EXISTS, JUST RELOAD
    # ===============================================
    else:
        print("[INFO] Per-class CSVs already exist.")
        for class_num in range(1, 10):
            df_dr = pd.read_csv(os.path.join(plots_dir, f"dose_rate_sphere_class_{class_num}.csv"))
            isot = df_dr.columns.tolist()[1:]
            drs = {iso: df_dr[iso].values for iso in isot}

            acc = 0.0
            for i in range(len(time_steps)):
                step = sum(drs[iso][i] for iso in isot) / 3600.0
                acc += step
                total_accumulated[i] += acc

    # 6) Save or reload total accumulated CSV
    total_path = os.path.join(plots_dir, total_csv)
    if not have_total:
        print("[INFO] Missing total accumulated CSV → generating now…")
        with open(total_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
            for tt, val in zip(time_steps, total_accumulated):
                w.writerow([tt, val])
    else:
        print("[INFO] Total accumulated CSV already exists. Reloading values…")
        df_tot = pd.read_csv(total_path)
        total_accumulated = df_tot["Accumulated_Dose (rads)"].values

    # 7) Plot total accumulated dose
    plot_accumulated_dose(
        total_accumulated,
        f"Total Accumulated Dose (Sphere) - Batch {batch_num}",
        f"total_accumulated_dose_sphere_batch_{batch_num}.png",
        out_dir=plots_dir
    )





=== Batch 1 — Dose Rates (Sphere) ===
[INFO] Per-class CSVs already exist.
[INFO] Total accumulated CSV already exists. Reloading values…

=== Batch 2 — Dose Rates (Sphere) ===
[INFO] Per-class CSVs already exist.
[INFO] Total accumulated CSV already exists. Reloading values…

=== Batch 3 — Dose Rates (Sphere) ===
[INFO] Per-class CSVs already exist.
[INFO] Total accumulated CSV already exists. Reloading values…

=== Batch 4 — Dose Rates (Sphere) ===
[INFO] Per-class CSVs already exist.
[INFO] Total accumulated CSV already exists. Reloading values…

=== Batch 5 — Dose Rates (Sphere) ===
[INFO] Per-class CSVs already exist.
[INFO] Total accumulated CSV already exists. Reloading values…

=== Batch 6 — Dose Rates (Sphere) ===
[INFO] Per-class CSVs already exist.
[INFO] Total accumulated CSV already exists. Reloading values…

=== Batch 7 — Dose Rates (Sphere) ===
[INFO] Per-class CSVs already exist.
[INFO] Total accumulated CSV already exists. Reloading values…


In [10]:
import os
import csv
import numpy as np
import pandas as pd
import scipy.special as sp

# ===============================================
# MODULE: CALCULATION OR LOADING OF DOSE RATE (Circular Base Method)
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Circular Base Method ===")

    # 1) Output directory and control volume
    output_directory = f"output_batch_{batch_num}"
    plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(plots_dir, exist_ok=True)
    V = V_values[batch_num]

    # 2) Reload ACTIVITIES (gamma energies come from JSON)
    all_class_activities = {}
    for class_num in range(1, 10):
        act_csv = os.path.join(output_directory, f"class_{class_num}_activities.csv")
        df_act = pd.read_csv(act_csv)
        isotopes = df_act.columns.tolist()[1:]
        all_class_activities[class_num] = {iso: df_act[iso].values for iso in isotopes}

    # 3) Parameters and total accumulator
    total_accumulated_csv = f"total_accumulated_dose_cb_batch_{batch_num}.csv"
    total_accumulated_path = os.path.join(plots_dir, total_accumulated_csv)

    total_accumulated_dose_cb = np.zeros(len(time_steps))
    h_base = 25  # cm (height of source)
    R_base = 15  # cm (radius of circular base)

    # 4) Check if per-class recalculation is needed
    expected = [f"dose_rate_circular_base_class_{i}.csv" for i in range(1, 10)]
    need_recalc = not all(fn in os.listdir(plots_dir) for fn in expected)

    # ===============================================
    # CALCULATE IF MISSING
    # ===============================================
    if need_recalc:
        print("[INFO] Missing per-class CSV files → calculating dose rates (Circular Base)…")

        for class_num, isotopes in class_inventories.items():
            acts = all_class_activities[class_num]
            dose_rates_cb = {iso: np.zeros(len(time_steps)) for iso in isotopes}
            total_per_timestep = np.zeros(len(time_steps))

            for i, t in enumerate(time_steps):
                sum_step = 0.0
                for iso in isotopes:
                    E = gamma_energies.get(iso, 0.0)
                    if E <= 0:
                        continue

                    A = acts.get(iso, np.zeros(len(time_steps)))[i]
                    if A <= 0:
                        continue

                    mu = mass_att(E)
                    sigma = max(1e-4, mass_ab(E))
                    if mu == 0:
                        continue

                    # Compute gamma flux using the circular base geometry
                    flux = gamma_flux_circular_base(A, R_base, h_base, mass_att, E)
                    dr = 5.77e-5 * flux * E * sigma
                    if np.isnan(dr):
                        continue

                    dose_rates_cb[iso][i] = dr
                    sum_step += dr / 3600.0  # convert to rads/h

                total_per_timestep[i] = sum_step

            # Accumulate dose per class
            acc = 0.0
            for i in range(len(time_steps)):
                acc += total_per_timestep[i]
                total_accumulated_dose_cb[i] += acc

            # Save per-class CSV
            out_csv = os.path.join(plots_dir, f"dose_rate_circular_base_class_{class_num}.csv")
            with open(out_csv, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + list(dose_rates_cb.keys()))
                for j, t in enumerate(time_steps):
                    w.writerow([t] + [dose_rates_cb[iso][j] for iso in isotopes])

            # Plot per-class dose rate
            plot_dose_rates(
                dose_rates_cb,
                f"Dose Rates (Circular Base) - Class {class_num}",
                f"dose_rates_circular_base_class_{class_num}.png",
                out_dir=plots_dir
            )

        print(f"[Batch {batch_num}] Per-class dose rates calculated.")

    # ===============================================
    # IF EXISTS, JUST RELOAD
    # ===============================================
    else:
        print("[INFO] Per-class CSV files exist → reloading dose rates from CSV…")

        for class_num in range(1, 10):
            df_cb = pd.read_csv(os.path.join(plots_dir, f"dose_rate_circular_base_class_{class_num}.csv"))
            isot = df_cb.columns.tolist()[1:]
            drs = {iso: df_cb[iso].values for iso in isot}

            acc = 0.0
            for i in range(len(time_steps)):
                step = sum(drs[iso][i] for iso in isot) / 3600.0
                acc += step
                total_accumulated_dose_cb[i] += acc

        print(f"[Batch {batch_num}] Dose rates successfully reloaded.")

    # ===============================================
    # SAVE OR RELOAD TOTAL ACCUMULATED DOSE
    # ===============================================
    if not os.path.exists(total_accumulated_path):
        print("[INFO] Missing total accumulated CSV → generating it now…")
        with open(total_accumulated_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
            for tt, val in zip(time_steps, total_accumulated_dose_cb):
                w.writerow([tt, val])
    else:
        print("[INFO] Total accumulated CSV already exists → reloading…")
        df_tot = pd.read_csv(total_accumulated_path)
        total_accumulated_dose_cb = df_tot["Accumulated_Dose (rads)"].values

    # ===============================================
    # PLOT TOTAL ACCUMULATED DOSE
    # ===============================================
    plot_accumulated_dose(
        total_accumulated_dose_cb,
        f"Total Accumulated Dose (Circular Base) - Batch {batch_num}",
        f"total_accumulated_dose_circular_base_batch_{batch_num}.png",
        out_dir=plots_dir
    )





=== Batch 1 — Circular Base Method ===
[INFO] Per-class CSV files exist → reloading dose rates from CSV…
[Batch 1] Dose rates successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 2 — Circular Base Method ===
[INFO] Per-class CSV files exist → reloading dose rates from CSV…
[Batch 2] Dose rates successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 3 — Circular Base Method ===
[INFO] Per-class CSV files exist → reloading dose rates from CSV…
[Batch 3] Dose rates successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 4 — Circular Base Method ===
[INFO] Per-class CSV files exist → reloading dose rates from CSV…
[Batch 4] Dose rates successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 5 — Circular Base Method ===
[INFO] Per-class CSV files exist → reloading dose rates from CSV…
[Batch 5] Dose rates successfully reloaded.
[INFO] Total accumulat

In [11]:
import os
import csv
import math
import numpy as np
import pandas as pd

# ===============================================
# MODULE: CALCULATION OR LOADING OF DOSE RATE (Circular Base + Shielding)
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Circular Base + Shielding ===")

    # 1) Output directory and control volume
    output_directory = f"output_batch_{batch_num}"
    plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(plots_dir, exist_ok=True)
    V = V_values[batch_num]

    # 2) Reload ACTIVITIES (gamma energies come from JSON)
    all_class_activities = {}
    for class_num in range(1, 10):
        df = pd.read_csv(os.path.join(output_directory, f"class_{class_num}_activities.csv"))
        isos = df.columns.tolist()[1:]
        all_class_activities[class_num] = {iso: df[iso].values for iso in isos}

    # 3) Geometrical parameters, material properties, and total accumulator
    total_acc_csv = f"total_accumulated_dose_shield_batch_{batch_num}.csv"
    total_acc_path = os.path.join(plots_dir, total_acc_csv)

    total_accumulated_dose_shield = np.zeros(len(time_steps))
    cylinder_radius = 15   # cm
    cylinder_height = 80   # cm
    R_shield = math.sqrt(2 * cylinder_radius * cylinder_height)
    density_polymer = 0.92  # g/cm³
    shield_thickness = 3    # cm

    def linear_att(E_keV, rho):
        """Linear attenuation coefficient as μ = μm(E) * ρ"""
        return calc_mu_m(E_keV) * rho

    # 4) Check if per-class recalculation is needed
    expected = [f"dose_rate_shield_class_{i}.csv" for i in range(1, 10)]
    need_recalc = not all(fn in os.listdir(plots_dir) for fn in expected)

    # ===============================================
    # CALCULATE IF MISSING
    # ===============================================
    if need_recalc:
        print("[INFO] Missing per-class CSVs → calculating Shielding…")

        for class_num, isotopes in class_inventories.items():
            acts = all_class_activities[class_num]
            dose_rates_shield = {iso: np.zeros(len(time_steps)) for iso in isotopes}
            timestep_dose = np.zeros(len(time_steps))

            for i, t in enumerate(time_steps):
                for iso in isotopes:
                    E = gamma_energies.get(iso, 0.0)
                    if E <= 0:
                        continue

                    A = acts.get(iso, np.zeros(len(time_steps)))[i]
                    if A <= 0:
                        continue

                    mu = mass_att(E)
                    sigma = max(1e-4, mass_ab(E))
                    if mu == 0:
                        continue

                    # Compute unshielded flux using circular base geometry
                    flux = gamma_flux_circular_base(A, R_shield, shield_thickness, mass_att, E)
                    dr = 5.77e-5 * flux * E * sigma

                    # Apply exponential attenuation through the shield
                    dr_sh = dr * np.exp(-linear_att(E * 1000, density_polymer) * shield_thickness)

                    if np.isnan(dr_sh):
                        continue

                    dose_rates_shield[iso][i] = dr_sh
                    timestep_dose[i] += dr_sh / 3600.0  # convert to rads/h

            # Accumulate per class
            acc = np.cumsum(timestep_dose)
            total_accumulated_dose_shield += acc

            # Save per-class CSV
            out_csv = os.path.join(plots_dir, f"dose_rate_shield_class_{class_num}.csv")
            with open(out_csv, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + list(dose_rates_shield.keys()))
                for j, t in enumerate(time_steps):
                    w.writerow([t] + [dose_rates_shield[iso][j] for iso in isotopes])

            # Plot per-class dose rates
            plot_dose_rates(
                dose_rates_shield,
                f"Dose Rates (Shielding) - Class {class_num}",
                f"dose_rate_shield_class_{class_num}.png",
                out_dir=plots_dir
            )
            print(f"  Class {class_num} → final accumulated dose: {acc[-1]:.2f} rads")

        print(f"[Batch {batch_num}] Shielding calculated.")

    # ===============================================
    # IF EXISTS, JUST RELOAD
    # ===============================================
    else:
        print("[INFO] Per-class CSVs exist → reloading Shielding from CSV…")

        for class_num in range(1, 10):
            df = pd.read_csv(os.path.join(plots_dir, f"dose_rate_shield_class_{class_num}.csv"))
            isos = df.columns.tolist()[1:]
            drs = {iso: df[iso].values for iso in isos}

            # Accumulate
            step = np.array([sum(drs[iso][i] for iso in isos) / 3600.0 for i in range(len(time_steps))])
            total_accumulated_dose_shield += np.cumsum(step)

        print(f"[Batch {batch_num}] Shielding successfully reloaded.")

    # ===============================================
    # SAVE OR RELOAD TOTAL ACCUMULATED DOSE
    # ===============================================
    if not os.path.exists(total_acc_path):
        print("[INFO] Missing total accumulated CSV → generating it now…")
        with open(total_acc_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
            for tt, val in zip(time_steps, total_accumulated_dose_shield):
                w.writerow([tt, val])
    else:
        print("[INFO] Total accumulated CSV already exists → reloading…")
        df_tot = pd.read_csv(total_acc_path)
        total_accumulated_dose_shield = df_tot["Accumulated_Dose (rads)"].values

    # ===============================================
    # PLOT TOTAL ACCUMULATED DOSE
    # ===============================================
    plot_accumulated_dose(
        total_accumulated_dose_shield,
        f"Total Accumulated Dose (Shielding) - Batch {batch_num}",
        f"total_accumulated_dose_shield_batch_{batch_num}.png",
        out_dir=plots_dir
    )





=== Batch 1 — Circular Base + Shielding ===
[INFO] Per-class CSVs exist → reloading Shielding from CSV…
[Batch 1] Shielding successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 2 — Circular Base + Shielding ===
[INFO] Per-class CSVs exist → reloading Shielding from CSV…
[Batch 2] Shielding successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 3 — Circular Base + Shielding ===
[INFO] Per-class CSVs exist → reloading Shielding from CSV…
[Batch 3] Shielding successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 4 — Circular Base + Shielding ===
[INFO] Per-class CSVs exist → reloading Shielding from CSV…
[Batch 4] Shielding successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 5 — Circular Base + Shielding ===
[INFO] Per-class CSVs exist → reloading Shielding from CSV…
[Batch 5] Shielding successfully reloaded.
[INFO] Total accumulated CSV alr

In [12]:
import os
import csv
import math
import numpy as np
import pandas as pd

# ===============================================
# MODULE: CALCULATION OR LOADING OF DOSE RATE (Point Source Method)
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Point Source Method ===")

    # 1) Output directory and plots
    output_directory = f"output_batch_{batch_num}"
    plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(plots_dir, exist_ok=True)

    # 2) Reload ACTIVITIES
    all_class_activities = {}
    for class_num in range(1, 10):
        df_act = pd.read_csv(os.path.join(output_directory, f"class_{class_num}_activities.csv"))
        isos = df_act.columns.tolist()[1:]  # skip time column
        all_class_activities[class_num] = {iso: df_act[iso].values for iso in isos}

    # 3) Parameters and total accumulator
    total_acc_csv = f"total_accumulated_dose_point_batch_{batch_num}.csv"
    total_acc_path = os.path.join(plots_dir, total_acc_csv)
    accumulated_dose_point = np.zeros(len(time_steps))
    h_point = 25  # cm (distance from source to receptor)
    density = 0.001  # g/cm³ (air density)

    def gamma_flux_point_source(activity, h, mass_att_func, E):
        """Gamma flux for a point source with attenuation (μ * ρ * h)."""
        mu = mass_att_func(E)
        if mu <= 0:
            return 0.0
        # 3.7e10 converts Ci → disintegrations/s if activity is in Ci
        return (3.7e10 * activity) / (4 * math.pi * h**2) * math.exp(-mu * density * h)

    # 4) Check for per-class CSVs
    expected = [f"dose_rate_point_source_class_{i}.csv" for i in range(1, 10)]
    need_recalc = not all(fn in os.listdir(plots_dir) for fn in expected)

    # ===============================================
    # CALCULATE IF MISSING
    # ===============================================
    if need_recalc:
        print("[INFO] Missing per-class CSVs → calculating Point Source…")

        for class_num, isotopes in class_inventories.items():
            # Extract isotope names if dict
            iso_names = list(isotopes.keys()) if isinstance(isotopes, dict) else list(isotopes)
            acts = all_class_activities[class_num]  # dict: iso -> activity array

            dose_rates_point = {iso: np.zeros(len(time_steps)) for iso in iso_names}
            timestep_dose = np.zeros(len(time_steps))  # sum at each timestep

            for i, t in enumerate(time_steps):
                for iso in iso_names:
                    E = gamma_energies.get(iso, 0.0)
                    if E <= 0:
                        continue

                    A = acts.get(iso, np.zeros(len(time_steps)))[i]
                    if A <= 0:
                        continue

                    sigma = max(1e-4, mass_ab(E))  # effective absorption coefficient
                    flux = gamma_flux_point_source(A, h_point, mass_att, E)
                    dr = 5.77e-5 * flux * E * sigma  # MeV → rads/s conversion

                    if np.isnan(dr):
                        continue

                    dose_rates_point[iso][i] = dr
                    timestep_dose[i] += dr / 3600.0  # convert to rads/h

            # Accumulate over time
            acc = np.cumsum(timestep_dose)
            accumulated_dose_point += acc

            # Save per-class CSV
            out_csv = os.path.join(plots_dir, f"dose_rate_point_source_class_{class_num}.csv")
            with open(out_csv, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + iso_names)
                for j, t in enumerate(time_steps):
                    w.writerow([t] + [dose_rates_point[iso][j] for iso in iso_names])

            # Plot per-class dose rates
            plot_dose_rates(
                dose_rates_point,
                f"Dose Rates (Point Source) - Class {class_num}",
                f"dose_rate_point_source_class_{class_num}.png",
                out_dir=plots_dir
            )

        print(f"[Batch {batch_num}] Point Source dose rates calculated.")

    # ===============================================
    # IF EXISTS, JUST RELOAD
    # ===============================================
    else:
        print("[INFO] Per-class CSVs already exist → reloading Point Source from CSV…")

        for class_num in range(1, 10):
            df_ps = pd.read_csv(os.path.join(plots_dir, f"dose_rate_point_source_class_{class_num}.csv"))
            isos = df_ps.columns.tolist()[1:]
            drs = {iso: df_ps[iso].values for iso in isos}

            steps = np.array([sum(drs[iso][i] for iso in isos) / 3600.0 for i in range(len(time_steps))])
            accumulated_dose_point += np.cumsum(steps)

        print(f"[Batch {batch_num}] Point Source successfully reloaded.")

    # ===============================================
    # SAVE OR RELOAD TOTAL ACCUMULATED DOSE
    # ===============================================
    if not os.path.exists(total_acc_path):
        print("[INFO] Missing total accumulated CSV → generating it now…")
        with open(total_acc_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
            for tt, val in zip(time_steps, accumulated_dose_point):
                w.writerow([tt, val])
    else:
        print("[INFO] Total accumulated CSV already exists → reloading…")
        df_tot = pd.read_csv(total_acc_path)
        accumulated_dose_point = df_tot["Accumulated_Dose (rads)"].values

    # ===============================================
    # PLOT TOTAL ACCUMULATED DOSE
    # ===============================================
    plot_accumulated_dose(
        accumulated_dose_point,
        f"Total Accumulated Dose (Point Source) - Batch {batch_num}",
        f"total_accumulated_dose_point_source_batch_{batch_num}.png",
        out_dir=plots_dir
    )




=== Batch 1 — Point Source Method ===
[INFO] Per-class CSVs already exist → reloading Point Source from CSV…
[Batch 1] Point Source successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 2 — Point Source Method ===
[INFO] Per-class CSVs already exist → reloading Point Source from CSV…
[Batch 2] Point Source successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 3 — Point Source Method ===
[INFO] Per-class CSVs already exist → reloading Point Source from CSV…
[Batch 3] Point Source successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 4 — Point Source Method ===
[INFO] Per-class CSVs already exist → reloading Point Source from CSV…
[Batch 4] Point Source successfully reloaded.
[INFO] Total accumulated CSV already exists → reloading…

=== Batch 5 — Point Source Method ===
[INFO] Per-class CSVs already exist → reloading Point Source from CSV…
[Batch 5] Point Source successfully re

In [13]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

# ===============================================
# MODULE: DAMAGE LEVELS (time-resolved calculation)
# ===============================================

eq_data = [
    (3600,    4e6),    # 1 hour
    (43200,   2e7),    # 12 hours
    (86400,   2.4e7),  # 1 day
    (864000,  4e7),    # 10 days
    (2592000, 5.5e7),  # 1 month
    (15552000,1.1e8),  # 6 months
    (31536000,1.5e8)   # 1 year
]

def level_from_ratio(r):
    if r < 1:
        return 1
    elif r < 4:
        return 2
    elif r < 7:
        return 3
    elif r < 10:
        return 4
    else:
        return 5

def calculate_damage_level(acc_dose, EQ, current_level):
    ratio = acc_dose / EQ if EQ > 0 else float('inf')
    lvl = level_from_ratio(ratio)
    return lvl, ratio, lvl

# ===============================================
# MAIN LOOP
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Damage Levels (time-resolved) ===")

    out_levels_dir = f"damage_levels_{batch_num}"
    csv_levels_dir = os.path.join(out_levels_dir, "csv_data")
    os.makedirs(csv_levels_dir, exist_ok=True)

    max_t = time_steps.max()
    segments = [(t, EQ) for t, EQ in eq_data if t <= max_t]

    dose_methods = {
        "Point Source":      "dose_rate_point_source_class_",
        "With Shield":       "dose_rate_shield_class_",
        "Sphere":            "dose_rate_sphere_class_",
        "Circular Base":     "dose_rate_circular_base_class_",
    }
    accumulated = {}

    # ===============================================
    # LOAD OR GENERATE TOTAL ACCUMULATED DOSES
    # ===============================================
    for label, prefix in dose_methods.items():
        plots_dir = os.path.join(f"output_batch_{batch_num}", "dose_rate_plots")
        total_csv = os.path.join(plots_dir, f"total_accumulated_{label.replace(' ', '_')}.csv")

        if os.path.exists(total_csv):
            data = np.loadtxt(total_csv, delimiter=',', skiprows=1, usecols=1)
            accumulated[label] = data
        else:
            cum_sum = np.zeros(len(time_steps))
            for class_num in range(1, 10):
                file_i = os.path.join(plots_dir, f"{prefix}{class_num}.csv")
                if not os.path.exists(file_i):
                    raise FileNotFoundError(f"Missing {file_i}")
                arr = np.loadtxt(file_i, delimiter=',', skiprows=1)[:, 1:]
                step = arr.sum(axis=1) / 3600.0
                cum = np.cumsum(step)
                cum_sum += cum
            with open(total_csv, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
                for t, val in zip(time_steps, cum_sum):
                    w.writerow([t, val])
            accumulated[label] = cum_sum

    # ===============================================
    # CALCULATE TIME-RESOLVED DAMAGE LEVELS
    # ===============================================
    for label, dose_arr in accumulated.items():
        key = label.replace(' ', '_').lower()
        csv_out = os.path.join(csv_levels_dir, f"{key}_dose_data.csv")
        damage_csv_out = os.path.join(csv_levels_dir, f"{key}_damage_levels.csv")
        png_out = os.path.join(out_levels_dir, f"damage_level_{key}.png")

        # --- Save accumulated dose data if not existing ---
        if not os.path.exists(csv_out):
            with open(csv_out, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time Step (s)", "Accumulated Dose (rads)"])
                for t, val in zip(time_steps, dose_arr):
                    w.writerow([t, val])

        # --- Initialize arrays ---
        damage_levels_over_time = np.zeros(len(time_steps), dtype=int)
        ratios_over_time = np.zeros(len(time_steps))
        EQ_used_over_time = np.zeros(len(time_steps))

        current_level = 1
        segment_idx = 0

        # --- Compute for each timestep ---
        for i, t in enumerate(time_steps):
            # If time exceeds the next segment boundary, move to next EQ
            while segment_idx + 1 < len(segments) and t >= segments[segment_idx + 1][0]:
                segment_idx += 1

            EQ = segments[segment_idx][1]
            EQ_used_over_time[i] = EQ
            acc_dose = dose_arr[i]

            level, ratio, _ = calculate_damage_level(acc_dose, EQ, current_level)
            current_level = level
            damage_levels_over_time[i] = level
            ratios_over_time[i] = ratio

        # --- Save detailed CSV ---
        with open(damage_csv_out, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated Dose (rads)", "EQ (rads)", "Ratio", "Damage Level"])
            for t, dose, EQ, ratio, lvl in zip(time_steps, dose_arr, EQ_used_over_time, ratios_over_time, damage_levels_over_time):
                w.writerow([t, dose, EQ, ratio, lvl])

        # --- Plot appearance preserved ---
        plt.figure(figsize=(8, 4))
        x_hours = time_steps / 3600.0
        plt.plot(x_hours, damage_levels_over_time, drawstyle='steps-post', linewidth=2.5, marker='o', markersize=4)
        plt.title(f"Damage Levels — {label} (Batch {batch_num})")
        plt.xlabel("Time (hours)")
        plt.ylabel("Damage Level (1–5)")
        plt.ylim(1, 6)
        plt.yticks([1, 2, 3, 4, 5])
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(png_out)
        plt.close()

        print(f"[{label}] Final Level = {damage_levels_over_time[-1]} | Max Ratio = {ratios_over_time.max():.2f}")

        # ============================================================
        # ADDITION: COPY SPHERE DAMAGE LEVELS CSVs TO CURRENT FOLDER
        # ============================================================
        if label == "Sphere" and batch_num in [1, 4, 5, 7]:
            dest_filename = f"sphere_damage_levels_{batch_num}.csv"
            dest_path = os.path.join(os.getcwd(), dest_filename)
            try:
                import shutil
                shutil.copyfile(damage_csv_out, dest_path)
                print(f"→ Copied Sphere CSV for Batch {batch_num} to current directory as '{dest_filename}'")
            except Exception as e:
                print(f"[WARNING] Could not copy Sphere CSV for Batch {batch_num}: {e}")

    print(f"[Batch {batch_num}] Damage Levels generated! ✅")






=== Batch 1 — Damage Levels (time-resolved) ===
[Point Source] Final Level = 5 | Max Ratio = 20300.53
[With Shield] Final Level = 5 | Max Ratio = 794500.29
[Sphere] Final Level = 5 | Max Ratio = 356.50
→ Copied Sphere CSV for Batch 1 to current directory as 'sphere_damage_levels_1.csv'
[Circular Base] Final Level = 5 | Max Ratio = 63971.40
[Batch 1] Damage Levels generated! ✅

=== Batch 2 — Damage Levels (time-resolved) ===
[Point Source] Final Level = 5 | Max Ratio = 16039.55
[With Shield] Final Level = 5 | Max Ratio = 628956.71
[Sphere] Final Level = 5 | Max Ratio = 311.85
[Circular Base] Final Level = 5 | Max Ratio = 50631.38
[Batch 2] Damage Levels generated! ✅

=== Batch 3 — Damage Levels (time-resolved) ===
[Point Source] Final Level = 5 | Max Ratio = 5394.32
[With Shield] Final Level = 5 | Max Ratio = 210613.35
[Sphere] Final Level = 5 | Max Ratio = 37.83
[Circular Base] Final Level = 5 | Max Ratio = 16962.23
[Batch 3] Damage Levels generated! ✅

=== Batch 4 — Damage Levels (ti

In [ ]:
import pandas as pd
import os
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np


# ============================
# 1) Import the level CSV files
# ============================
df1 = pd.read_csv("sphere_damage_levels_1.csv")
df4 = pd.read_csv("sphere_damage_levels_4.csv")
df5 = pd.read_csv("sphere_damage_levels_5.csv")
df7 = pd.read_csv("sphere_damage_levels_7.csv")

# ===================================
# 2) Instrumentation to location definition
# ===================================
instrumentation_by_location = {
    "WETWELL": ["Suppression Pool Thermocouples"],
    "DRYWELL": [
        "SRV Position Indicators",
        "Radiation Detectors in Containment",
        "Boron Injection Pressure Transmitters",
        "Boron Tank Level Transmitter",
        "Drywell Pressure Transmitters",
        "Containment Pressure Transmitters",
        "Containment Thermocouples",
        "Vessel Level (Wide Range)",
        "Vessel Level (Fuel Range)",
        "Vessel Pressure Transmitters",
        "Feedwater Flow Transmitters",
        "Drywell Thermocouples",
        "Radiation Detectors in Drywell",
        "Control Rods Position",
    ],
    "LOWER_HEAD": ["Vessel Thermocouples (Lower Head)", "FW Thermocouples"],
    "DOME": ["Vessel Thermocouples (Upper Head)"],
    "RECIRCULATION_PUMPS": [
        "Suction Pipes Thermocouples",
        "Recirculation Flow Transmitters",
        "Recirculation Pressure Transmitters",
    ],
    "ANNULUS": ["Average Reactor Power Monitors"],
    "AUXILIARY_BUILDING": [
        "Containment Hydrogen Concentration Analyzer",
        "Drywell Hydrogen Concentration Analyzer",
        "Feedwater Pressure Transmitters",
        "HPCS Flow Transmitter",
        "Suppression Pool Level Transmitters",
    ],
    "FUEL_BUILDING": [],
}

# =================================
# 3) Link the location to level CSV files
# =================================
level_map = {
    "WETWELL": 4,
    "DRYWELL": 7,
    "LOWER_HEAD": 1,
    "DOME": 1,
    "RECIRCULATION_PUMPS": 1,
    "ANNULUS": 1,
    "AUXILIARY_BUILDING": 5,
    "FUEL_BUILDING": 5,
}

# ==============================
# 4) CSV to dict conversion
# ==============================
level_df_map = {
    1: df1.to_dict(orient="records"),
    4: df4.to_dict(orient="records"),
    5: df5.to_dict(orient="records"),
    7: df7.to_dict(orient="records"),
}

# ===================================================
# 5) Link instrument with damage level
# ===================================================
damage_by_instrument = {}
for location, instruments in instrumentation_by_location.items():
    level = level_map[location]
    damage_data = level_df_map[level]
    for instr in instruments:
        damage_by_instrument[instr] = damage_data

# =========================================================
# 6) Actions and Guides
# =========================================================
actions_library = {
    "G1_BREACH_IN_VESSEL": {
        "TEMPERATURE DRYWELL": [
            "Drywell Thermocouples",
            "Containment Thermocouples",
            "Suppression Pool Thermocouples"
        ],
        "PRESSURE DRYWELL": [
            "Drywell Pressure Transmitters",
            "Containment Pressure Transmitters"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell",
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G2_BORON_INJECTION": {
        "LEVEL BORON TANK": [
            "Boron Tank Level Transmitter",
            "Boron Injection Pressure Transmitters"
        ],
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "TEMPERATURE RPV": [
            "Vessel Thermocouples (Upper Head)",
            "Vessel Thermocouples (Lower Head)",
            "FW Thermocouples",
            "Suction Pipes Thermocouples"
        ]
    },
    "G3_SPRAY": {
        "PRESSURE CONTAINMENT": [
            "Containment Pressure Transmitters",
            "Drywell Pressure Transmitters"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G4_VENTING": {
        "PRESSURE CONTAINMENT": [
            "Containment Pressure Transmitters",
            "Drywell Pressure Transmitters"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ],
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
    },
    "G5_INJECTION_RPV": {
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "TEMPERATURE RPV": [
            "Vessel Thermocouples (Upper Head)",
            "Vessel Thermocouples (Lower Head)",
            "FW Thermocouples",
            "Suction Pipes Thermocouples"
        ]
    },
    "G6_INJECTION_RPV": {
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "TEMPERATURE RPV": [
            "Vessel Thermocouples (Upper Head)",
            "Vessel Thermocouples (Lower Head)",
            "FW Thermocouples",
            "Suction Pipes Thermocouples"
        ]
    },
    "G7_INJECTION_PC_UP_TO_TAF": {
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "PRESSURE SP": [
            "Suppression Pool Thermocouples",
            "Containment Pressure Transmitters",
            "Drywell Pressure Transmitters"
        ]
    },
    "G9_SUPPORT_VENTING_VESSEL": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G10_EVIDENCES_LOCA": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE DRYWELL": [
            "Drywell Pressure Transmitters",
            "Containment Pressure Transmitters"
        ],
        "SUPPRESSION POOL TEMPERATURE": [
            "Suppression Pool Thermocouples",
            "Drywell Thermocouples",
            "Containment Thermocouples"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G12_FLOOD_RPV_UP_TO_TAF": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "SRV POSITION": ["SRV Position Indicators"],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ]
    },
    "G13_SUPPORT_VENTING_RPV": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "SRV POSITION": ["SRV Position Indicators"],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G14_DETERMINE_INVESSEL_RETENTION": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "TEMPERATURE RPV": [
            "Vessel Thermocouples (Upper Head)",
            "Vessel Thermocouples (Lower Head)",
            "FW Thermocouples",
            "Suction Pipes Thermocouples"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ]
    },
    "GP": {
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "SRV POSITION": ["SRV Position Indicators"],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "PRESSURE DRYWELL": [
            "Drywell Pressure Transmitters",
            "Containment Pressure Transmitters"
        ]
    },
    "GQ": {
        "REACTOR POWER": [
            "Average Reactor Power Monitors",
            "Control Rods Position"
        ]
    },
    "GT/D": {
        "TEMPERATURE DRYWELL": [
            "Drywell Thermocouples",
            "Containment Thermocouples",
            "Suppression Pool Thermocouples"
        ]
    },
    "GT/PC": {
        "TEMPERATURE CONTAINMENT": [
            "Containment Thermocouples",
            "Drywell Thermocouples",
            "Suppression Pool Thermocouples"
        ]
    },
    "GT/SP": {
        "TEMPERATURE SUPPRESSION POOL": [
            "Suppression Pool Thermocouples",
            "Drywell Thermocouples",
            "Containment Thermocouples"
        ]
    },
    "GH/PC": {
        "PRESSURE CONTAINMENT": [
            "Containment Pressure Transmitters",
            "Containment Pressure Transmitter (Auxiliary Building)",
            "Drywell Pressure Transmitters",
            "Drywell Pressure Transmitter (Auxiliary Building)"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "GR/PC": {
        "PRESSURE CONTAINMENT": [
            "Containment Pressure Transmitters",
            "Containment Pressure Transmitter (Auxiliary Building)",
            "Drywell Pressure Transmitters",
            "Drywell Pressure Transmitter (Auxiliary Building)"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ]
    },
    "GL/SP": {
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ]
    }
}

actions_short = { k.split("_")[0]: v for k,v in actions_library.items() }


# SAMG1 and SAMG2
SAMG1 = {
    "RC_F1": ["G1", "G3", "G4", "G5", "G6", "G7"],
    "RC_F2": ["G12", "G3", "G4", "G7", "G9", "G13"],
    "RC_F3": ["G3", "G4", "G12"],
    "RC_F4": ["G14", "G3", "G4", "G6"],
    "RC_F5": ["G3", "G4", "G5", "G7", "G14"],
    "RC_Q": ["G1", "G2", "GQ"],
    "RC_P": ["GP"],
}

SAMG2 = {
    "DW_T": ["GT/D"],
    "CN_T": ["GT/PC"],
    "SP_T": ["GT/SP"],
    "PC_P": ["G4"],
    "PC_H": ["GH/PC"],
    "PC_R": ["GR/PC"],
    "SP_L": ["GL/SP"],
}

# =========================================================
# 7) Utilities functions
# =========================================================
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def _detect_interval_key(sample_records):
    if not sample_records:
        return "Interval"
    keys = list(sample_records[0].keys())
    lowmap = {k.lower(): k for k in keys}
    for cand in ["intervalo", "interval"]:
        if cand in lowmap:
            return lowmap[cand]
    for k in keys:
        if "interval" in k.lower():
            return k
    return keys[0]

def _detect_level_key(sample_records):
    if not sample_records:
        return "Damage Level"
    keys = list(sample_records[0].keys())
    lowmap = {k.lower(): k for k in keys}
    for cand in ["nivel de daño", "nivel_de_daño", "damage level", "damage_level", "damage"]:
        if cand in lowmap:
            return lowmap[cand]
    for k in keys:
        kl = k.lower()
        if "damage" in kl or "daño" in kl:
            return k
    return keys[-1]

# =========================================================
# 8) Functions to build dataframes and states
# =========================================================
def _levels_dataframe_for_measure(instrs, damage_by_instrument, fallback_df_records):
    sample = next((damage_by_instrument[i] for i in instrs if i in damage_by_instrument), fallback_df_records)
    interval_key = _detect_interval_key(sample)
    level_key = _detect_level_key(sample)
    intervals = [r[interval_key] for r in sample]
    data = {interval_key: intervals}
    for instr in instrs:
        recs = damage_by_instrument.get(instr)
        if not recs:
            data[instr] = [1] * len(intervals)
            continue
        vals_raw = [rec.get(level_key, None) for rec in recs]
        vals = [int(v) if v is not None and str(v).strip() != "" else 1 for v in vals_raw]
        if len(vals) < len(intervals):
            vals += [vals[-1] if vals else 1] * (len(intervals) - len(vals))
        elif len(vals) > len(intervals):
            vals = vals[:len(intervals)]
        data[instr] = vals
    df = pd.DataFrame(data)
    return interval_key, level_key, intervals, df

def _availability_dataframe_for_measure(instrs, damage_by_instrument, fallback_df_records):
    sample = next((damage_by_instrument[i] for i in instrs if i in damage_by_instrument), fallback_df_records)
    interval_key = _detect_interval_key(sample)
    level_key = _detect_level_key(sample)
    intervals = [r[interval_key] for r in sample]
    all_levels = []
    for instr in instrs:
        recs = damage_by_instrument.get(instr)
        if not recs:
            all_levels.append([1] * len(intervals))
        else:
            vals_raw = [rec.get(level_key, None) for rec in recs]
            vals = [int(v) if v is not None and str(v).strip() != "" else 1 for v in vals_raw]
            if len(vals) < len(intervals):
                vals += [vals[-1] if vals else 1] * (len(intervals) - len(vals))
            elif len(vals) > len(intervals):
                vals = vals[:len(intervals)]
            all_levels.append(vals)
    states, codes = [], []
    for i in range(len(intervals)):
        lvls_t = [levels[i] for levels in all_levels] if all_levels else []
        st = _classify_state_from_levels(lvls_t)
        states.append(st)
        codes.append(state_map[st])
    df = pd.DataFrame({interval_key: intervals, "state": states, "state_code": codes})
    return interval_key, intervals, df

# =========================================================
# 9) Plotting functions
# =========================================================
def plot_damage(instruments, damage_by_instrument, save_path, title):
    # Tomamos una muestra para detectar claves
    sample = next((damage_by_instrument[i] for i in instruments if i in damage_by_instrument), None)
    if not sample:
        return

    interval_key = _detect_interval_key(sample)   # aquí detectará "Time (s)"
    level_key    = _detect_level_key(sample)      # aquí detectará "Damage Level"

    fig, ax = plt.subplots(figsize=(14, 7), dpi=120)
    fig.patch.set_facecolor("#001F3F")
    ax.set_facecolor("#001F3F")

    # Paleta sencilla y sin marcadores (muchos puntos → carísimo)
    colors = plt.cm.tab10.colors

    for idx, instr in enumerate(instruments):
        recs = damage_by_instrument.get(instr)
        if not recs:
            continue

        # x = tiempo real; y = nivel de daño (1..5)
        x = np.asarray([float(r[interval_key]) for r in recs])
        y = np.asarray([int(r.get(level_key, 1)) for r in recs], dtype=int)

        # Decimado automático si hay demasiados puntos (p. ej. > 3000)
        max_points = 3000
        n = len(x)
        if n > max_points:
            step = int(np.ceil(n / max_points))
            x = x[::step]
            y = y[::step]

        ax.plot(
            x, y,
            linewidth=1.5,
            label=instr,
            color=colors[idx % len(colors)]
        )

    ax.set_xlabel("Time (s)", color="white", fontsize=14)
    ax.set_ylabel("Damage Level", color="white", fontsize=14)
    ax.set_yticks([1, 2, 3, 4, 5])
    ax.set_yticklabels([1, 2, 3, 4, 5], color="white", fontsize=12)
    ax.set_title(title, color="white", fontsize=20, pad=16)

    ax.tick_params(colors="white", labelsize=12)
    ax.grid(color="white", linestyle="--", alpha=0.2, linewidth=1)
    for spine in ax.spines.values():
        spine.set_edgecolor("white")

    # Leyenda compacta
    leg = ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.12),
                    ncol=min(len(instruments), 3), frameon=False, fontsize=11)
    for txt in (leg.get_texts() if leg else []):
        txt.set_color("white")

    plt.tight_layout()
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=150)
    plt.close(fig)


# =========================================================
# 10) Availability functions
# =========================================================
state_map = {"Good": 0, "Careful": 1, "Warning": 2, "Blind": 3}
zone_colors = {0: "#006400", 1: "#CCCC00", 2: "#FF8C00", 3: "#8B0000"}

def plot_availability(measure, intervals, states, save_path):
    # Convertimos estados a códigos 0..3
    values = np.asarray([state_map[s] for s in states], dtype=int)
    x = np.asarray(intervals, dtype=float)  # aquí será "Time (s)"

    # Decimado automático (mismo criterio)
    max_points = 3000
    n = len(x)
    if n > max_points:
        step = int(np.ceil(n / max_points))
        x = x[::step]
        values = values[::step]

    fig, ax = plt.subplots(figsize=(14, 6), dpi=120)
    fig.patch.set_facecolor("#001F3F")
    ax.set_facecolor("#001F3F")

    # Bandas de fondo por zona
    for lvl, col in zone_colors.items():
        ax.axhspan(lvl, lvl + 1, color=col, alpha=0.25)

    # Línea simple (sin marcadores)
    ax.plot(x, values, linewidth=2, color="#00CED1")

    ax.set_xlabel("Time (s)", color="white", fontsize=14)
    ax.set_ylabel("State", color="white", fontsize=14)
    ax.set_yticks(list(state_map.values()))
    ax.set_yticklabels(list(state_map.keys()), color="white", fontsize=12)
    ax.set_title(f"{measure} Availability", color="white", fontsize=20, pad=16)

    ax.tick_params(colors="white", labelsize=12)
    ax.grid(color="white", linestyle="--", alpha=0.2, linewidth=1)
    for spine in ax.spines.values():
        spine.set_edgecolor("white")

    plt.tight_layout()
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=150)
    plt.close(fig)


def _classify_state_from_levels(levels):
    if not levels:
        return "Blind"
    lvls = [int(x) for x in levels if x is not None]
    if all(l == 1 for l in lvls):
        return "Good"
    if all(l == 5 for l in lvls):
        return "Blind"
    if any(l in (3, 4) for l in lvls):
        return "Warning"
    if any(l == 2 for l in lvls) and all(l <= 2 for l in lvls):
        return "Careful"
    return "Warning"

# =========================================================
# 11) CSV and graphs generators
# =========================================================
def generate_graphs(samg_name, samg_dict, actions_map, damage_by_instrument):
    create_directory(samg_name)
    for subguide, codes in samg_dict.items():
        path_sub = os.path.join(samg_name, subguide)
        create_directory(path_sub)
        for code in codes:
            if code not in actions_map:
                continue
            path_code = os.path.join(path_sub, code)
            create_directory(path_code)
            for measure, instrs in actions_map[code].items():
                base = measure.replace(" ", "_").replace("/", "_")
                interval_key, level_key, intervals, df_levels = _levels_dataframe_for_measure(
                    instrs, damage_by_instrument, df1.to_dict(orient="records")
                )
                df_levels.to_csv(os.path.join(path_code, base + ".csv"), index=False, encoding="utf-8")
                plot_damage(instrs, damage_by_instrument, os.path.join(path_code, base + ".png"), measure)

def generate_availability(samg_name, samg_dict, actions_map, damage_by_instrument):
    base = f"Availability_{samg_name}"
    create_directory(base)
    for subguide, codes in samg_dict.items():
        path_sub = os.path.join(base, subguide)
        create_directory(path_sub)
        for code in codes:
            if code not in actions_map:
                continue
            path_code = os.path.join(path_sub, code)
            create_directory(path_code)
            for measure, instrs in actions_map[code].items():
                safe = measure.replace(" ", "_").replace("/", "_")
                interval_key, intervals, df_av = _availability_dataframe_for_measure(
                    instrs, damage_by_instrument, df1.to_dict(orient="records")
                )
                df_av.to_csv(os.path.join(path_code, safe + "_availability.csv"), index=False, encoding="utf-8")
                plot_availability(measure, intervals, df_av["state"].tolist(), os.path.join(path_code, safe + "_availability.png"))

# =========================================================
# 12) Execution
# =========================================================
generate_graphs("SAMG1_R", SAMG1, actions_short, damage_by_instrument)
generate_graphs("SAMG2_R", SAMG2, actions_short, damage_by_instrument)
generate_availability("SAMG1_R", SAMG1, actions_short, damage_by_instrument)
generate_availability("SAMG2_R", SAMG2, actions_short, damage_by_instrument)


In [15]:
import numpy as np
from pypost.codes.melcor import MELCOR

file_path_melcor = "MC_Step.ptf"   
file_index_melcor = MELCOR.openPlotFile(file_path_melcor)

def cargar(var_code: str):
    data = np.array(MELCOR.getData(file_index_melcor, var_code))
    if data.ndim != 2 or data.shape[1] < 2:
        raise RuntimeError(f"Unexpected format for {var_code}: shape={data.shape}")
    return data[:, 0], data[:, 1]

# =========================
# Import of MELCOR variables
# =========================
tiempo_s, TempVapor_Wetwell = cargar("CVH-TVAP_305")

def cargar_misma_malla(var_code: str):
    t, v = cargar(var_code)
    if not np.array_equal(tiempo_s, t):
        if np.allclose(tiempo_s, t, rtol=0, atol=1e-12):
            return v
        raise ValueError(f"La malla de tiempo en {var_code} no coincide con la referencia.")
    return v

Presion_Wetwell          = cargar_misma_malla("CVH-P_305")
TempVapor_Drywell        = cargar_misma_malla("CVH-TVAP_201")
Presion_Drywell          = cargar_misma_malla("CVH-P_201")
TempVapor_CabezaSuperior = cargar_misma_malla("CVH-TVAP_160")
Presion_CabezaSuperior   = cargar_misma_malla("CVH-P_160")
TempVapor_CabezaInferior = cargar_misma_malla("CVH-TVAP_120")
Presion_CabezaInferior   = cargar_misma_malla("CVH-P_120")
TempVapor_BombaRecir     = cargar_misma_malla("CVH-TVAP_106")
Presion_BombaRecir       = cargar_misma_malla("CVH-P_106")

# =========================
# VARIABLES CONSTANTES
# =========================
EQ_PRESSURE = 410000   # Pa
EQ_TEMP = 422          # K

# =========================
# CRITERIO DE FALLO DE VASIJA
# =========================
VESSEL_FAILURE = Presion_BombaRecir < 6_000_000  # Boolean array

# =========================
# DICCIONARIO PARA USO POSTERIOR
# =========================
variables_melcor = {
    "tiempo_s": tiempo_s,
    "TempVapor_Wetwell":        TempVapor_Wetwell,
    "Presion_Wetwell":          Presion_Wetwell,
    "TempVapor_Drywell":        TempVapor_Drywell,
    "Presion_Drywell":          Presion_Drywell,
    "TempVapor_CabezaSuperior": TempVapor_CabezaSuperior,
    "Presion_CabezaSuperior":   Presion_CabezaSuperior,
    "TempVapor_CabezaInferior": TempVapor_CabezaInferior,
    "Presion_CabezaInferior":   Presion_CabezaInferior,
    "TempVapor_BombaRecir":     TempVapor_BombaRecir,
    "Presion_BombaRecir":       Presion_BombaRecir,
    "EQ_PRESSURE":              EQ_PRESSURE,
    "EQ_TEMP":                  EQ_TEMP,
    "VESSEL_FAILURE":           VESSEL_FAILURE
}

# =========================
# CONTENEDOR: niveles por instrumento
# =========================
niveles_instrumentos = {}  # { nombre_instrumento: ndarray[int] (niveles 1..5 en cada t) }

def guardar_nivel_instrumento(nombre: str, level_array: np.ndarray):
    """
    Guarda la serie de niveles (1..5) a lo largo del tiempo para un instrumento.
    """
    if level_array.shape != tiempo_s.shape:
        raise ValueError(f"Longitud de niveles de '{nombre}' no coincide con tiempo.")
    niveles_instrumentos[nombre] = level_array.astype(int, copy=False)

# =========================
# INSTRUMENTO: Suppression Pool Thermocouples (niveles 1–5)
# Reglas interpretadas:
# 1: 273–422 K (nominal)  [Estado base; no fuerza 1 si hay niveles superiores]
# 2: CEC>EQ => T_wet > EQ_TEMP  OR  P_wet > EQ_PRESSURE
# 3: T_wet > 673 K
# 5: T_wet > 1530 K
# (Nivel 4 aún sin criterio)
# =========================
# HISTÉRESIS TEMPORAL (30 min para bajar de nivel)
# =========================
def aplicar_histeresis(tiempo, niveles_condiciones, retardo_bajada_s):
    """
    Aplica histéresis temporal a una serie de niveles (1..5).
    - Subida inmediata al nivel mayor si se detecta.
    - Bajada solo si han pasado retardo_bajada_s desde que se cumplen condiciones de nivel menor.
    """
    niveles_final = np.zeros_like(niveles_condiciones)
    niveles_final[0] = niveles_condiciones[0]
    tiempo_ultimo_cambio = tiempo[0]

    for i in range(1, len(tiempo)):
        nivel_actual = niveles_final[i-1]
        nivel_cond = niveles_condiciones[i]

        if nivel_cond > nivel_actual:
            # Subida inmediata
            niveles_final[i] = nivel_cond
            tiempo_ultimo_cambio = tiempo[i]
        elif nivel_cond < nivel_actual:
            # Baja solo si ha pasado el retardo
            if tiempo[i] - tiempo_ultimo_cambio >= retardo_bajada_s:
                niveles_final[i] = nivel_cond
                tiempo_ultimo_cambio = tiempo[i]
            else:
                niveles_final[i] = nivel_actual
        else:
            # Mismo nivel
            niveles_final[i] = nivel_actual

    return niveles_final

# =========================
# INSTRUMENTO: Suppression Pool Thermocouples con histéresis
# =========================
T_wet = TempVapor_Wetwell
P_wet = Presion_Wetwell

# Niveles por condiciones instantáneas
niveles_cond = np.ones_like(tiempo_s, dtype=int)
niveles_cond[(T_wet > EQ_TEMP) | (P_wet > EQ_PRESSURE)] = 2
niveles_cond[T_wet > 673.0] = 3
niveles_cond[T_wet > 1530.0] = 5

# Aplicar histéresis de 1800 s para bajar
SuppressionPool_TC_Level = aplicar_histeresis(tiempo_s, niveles_cond, 1800.0)

# Guardar
guardar_nivel_instrumento("SuppressionPoolThermocouples", SuppressionPool_TC_Level)

# =========================
# INSTRUMENTO: SRV Position Indicators (con T y P del DRYWELL)
# =========================

# Niveles por condiciones instantáneas (1..5)
niveles_cond_srv = np.ones_like(tiempo_s, dtype=int)

# Nivel 2: CEC>EQ usando Drywell
niveles_cond_srv[(TempVapor_Drywell > EQ_TEMP) | (Presion_Drywell > EQ_PRESSURE)] = 2

# Nivel 5: Vessel Failure (ya definido con Presion_BombaRecir < 6 MPa)
niveles_cond_srv[VESSEL_FAILURE] = 5

# Aplicar histéresis de 1800 s para bajar
SRV_PositionIndicators_Level = aplicar_histeresis(tiempo_s, niveles_cond_srv, 1800.0)

# Guardar serie de niveles en el contenedor general
guardar_nivel_instrumento("SRVPositionIndicators", SRV_PositionIndicators_Level)

# =========================
# INSTRUMENTO: Radiation Detectors in Containment (Drywell)
# =========================
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell

# Niveles por condiciones instantáneas (1..5)
niveles_cond_rad = np.ones_like(tiempo_s, dtype=int)

# Aplicar de menor a mayor gravedad; los superiores sobreescriben a los inferiores
# Nivel 2: CEC>EQ (OR en T o P)
niveles_cond_rad[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2

# Nivel 3: 1.5*CEC>EQ
niveles_cond_rad[(T_dw > 1.5*EQ_TEMP) | (P_dw > 1.5*EQ_PRESSURE)] = 3

# Nivel 5: 2*CEC>EQ
niveles_cond_rad[(T_dw > 2.0*EQ_TEMP) | (P_dw > 2.0*EQ_PRESSURE)] = 5

# Histéresis: 1800 s para bajar de nivel
RadiationDetectorsContainment_Level = aplicar_histeresis(tiempo_s, niveles_cond_rad, 1800.0)

# Guardar serie de niveles
guardar_nivel_instrumento("RadiationDetectorsContainment", RadiationDetectorsContainment_Level)

# =========================
# INSTRUMENTO: Boron Injection Pressure Transmitters (como presión en contención)
# =========================
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell

# Niveles por condiciones instantáneas (1..5)
niveles_cond_bipt = np.ones_like(tiempo_s, dtype=int)

# Nivel 2: CEC>EQ (OR en T o P)
niveles_cond_bipt[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2

# Nivel 3: 1.5*CEC>EQ
niveles_cond_bipt[(T_dw > 1.5*EQ_TEMP) | (P_dw > 1.5*EQ_PRESSURE)] = 3

# Nivel 5: 2*CEC>EQ
niveles_cond_bipt[(T_dw > 2.0*EQ_TEMP) | (P_dw > 2.0*EQ_PRESSURE)] = 5

# Histéresis: 1800 s para bajar de nivel
BoronInjectionPressureTransmitters_Level = aplicar_histeresis(tiempo_s, niveles_cond_bipt, 1800.0)

# Guardar serie de niveles
guardar_nivel_instrumento("BoronInjectionPressureTransmitters", BoronInjectionPressureTransmitters_Level)

# =========================
# INSTRUMENTO: Boron Tank Level Transmitter (Drywell)
# =========================
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell

# Helper: duración continua de una condición booleana (segundos)
def duracion_continua(tiempo, mask_bool):
    dur = np.zeros_like(tiempo, dtype=float)
    for i in range(1, len(tiempo)):
        if mask_bool[i]:
            dur[i] = (dur[i-1] + (tiempo[i] - tiempo[i-1])) if mask_bool[i-1] else 0.0
        else:
            dur[i] = 0.0
    return dur

# Niveles por condiciones instantáneas (1..5)
niveles_cond_btl = np.ones_like(tiempo_s, dtype=int)

# Nivel 2: CEC>EQ (OR en T o P del Drywell)
niveles_cond_btl[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2

# Nivel 3: 30 min después de Tcont>422 K (condición mantenida de forma continua)
mask_T422 = T_dw > 422.0
dur_T422 = duracion_continua(tiempo_s, mask_T422)
niveles_cond_btl[dur_T422 >= 1800.0] = 3

# Nivel 5: Tcont > 1530 K
niveles_cond_btl[T_dw > 1530.0] = 5

# Aplicar histéresis de 1800 s para bajar de nivel
BoronTankLevelTransmitter_Level = aplicar_histeresis(tiempo_s, niveles_cond_btl, 1800.0)

# Guardar serie de niveles
guardar_nivel_instrumento("BoronTankLevelTransmitter", BoronTankLevelTransmitter_Level)

# =========================
# INSTRUMENTO: Drywell Pressure Transmitters
# =========================
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell

niveles_cond_dpt = np.ones_like(tiempo_s, dtype=int)
# Nivel 2: CEC>EQ (OR)
niveles_cond_dpt[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2
# Nivel 3: 1.5*CEC>EQ
niveles_cond_dpt[(T_dw > 1.5*EQ_TEMP) | (P_dw > 1.5*EQ_PRESSURE)] = 3
# Nivel 5: 2*CEC>EQ
niveles_cond_dpt[(T_dw > 2.0*EQ_TEMP) | (P_dw > 2.0*EQ_PRESSURE)] = 5

DrywellPressureTransmitters_Level = aplicar_histeresis(tiempo_s, niveles_cond_dpt, 1800.0)
guardar_nivel_instrumento("DrywellPressureTransmitters", DrywellPressureTransmitters_Level)

# =========================
# INSTRUMENTO: Containment Pressure Transmitters
# =========================
niveles_cond_cpt = np.ones_like(tiempo_s, dtype=int)
# Nivel 2: CEC>EQ (OR)
niveles_cond_cpt[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2
# Nivel 3: 1.5*CEC>EQ
niveles_cond_cpt[(T_dw > 1.5*EQ_TEMP) | (P_dw > 1.5*EQ_PRESSURE)] = 3
# Nivel 5: 2*CEC>EQ
niveles_cond_cpt[(T_dw > 2.0*EQ_TEMP) | (P_dw > 2.0*EQ_PRESSURE)] = 5

ContainmentPressureTransmitters_Level = aplicar_histeresis(tiempo_s, niveles_cond_cpt, 1800.0)
guardar_nivel_instrumento("ContainmentPressureTransmitters", ContainmentPressureTransmitters_Level)

# =========================
# INSTRUMENTO: Containment Thermocouples (Drywell)
# =========================
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell

# Niveles por condiciones instantáneas (1..5)
niveles_cond_ctc = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto (base)

# Nivel 2: CEC>EQ (OR en T o P)
niveles_cond_ctc[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2

# Nivel 3: Tcont > 673 K
niveles_cond_ctc[T_dw > 673.0] = 3

# Nivel 5: Tcont > 1530 K
niveles_cond_ctc[T_dw > 1530.0] = 5

# Histéresis: 1800 s para bajar de nivel
ContainmentThermocouples_Level = aplicar_histeresis(tiempo_s, niveles_cond_ctc, 1800.0)

# Guardar serie de niveles
guardar_nivel_instrumento("ContainmentThermocouples", ContainmentThermocouples_Level)

# =========================
# INSTRUMENTO: Vessel Level (Wide Range)
# =========================
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell

niveles_cond_vlwr = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto

# Nivel 2: CEC>EQ (OR en T o P)
niveles_cond_vlwr[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2

# Nivel 3: 30 min continuos tras Tcont>422 K
mask_T422_dw = T_dw > 422.0
dur_T422_dw  = duracion_continua(tiempo_s, mask_T422_dw)
niveles_cond_vlwr[dur_T422_dw >= 1800.0] = 3

# Nivel 5: Vessel Failure (omite nivel 4)
niveles_cond_vlwr[VESSEL_FAILURE] = 5

VesselLevelWideRange_Level = aplicar_histeresis(tiempo_s, niveles_cond_vlwr, 1800.0)
guardar_nivel_instrumento("VesselLevelWideRange", VesselLevelWideRange_Level)

# =========================
# INSTRUMENTO: Vessel Level (Fuel Range)
# =========================
niveles_cond_vlfr = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto

# Nivel 2: CEC>EQ (OR en T o P)
niveles_cond_vlfr[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2

# Nivel 3: 30 min continuos tras Tcont>422 K
niveles_cond_vlfr[dur_T422_dw >= 1800.0] = 3  # reutilizamos dur_T422_dw calculado arriba

# Nivel 5: Vessel Failure (omite nivel 4)
niveles_cond_vlfr[VESSEL_FAILURE] = 5

VesselLevelFuelRange_Level = aplicar_histeresis(tiempo_s, niveles_cond_vlfr, 1800.0)
guardar_nivel_instrumento("VesselLevelFuelRange", VesselLevelFuelRange_Level)

# =========================
# INSTRUMENTO: Vessel Pressure Transmitters
# =========================
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell
PRCS = Presion_BombaRecir  # presión de bomba de recirculación

# Duración continua de T_dw > 422 K (para el nivel 3)
mask_T422_dw = T_dw > 422.0
dur_T422_dw  = duracion_continua(tiempo_s, mask_T422_dw)

# Niveles por condiciones instantáneas (1..5)
niveles_cond_vpt = np.ones_like(tiempo_s, dtype=int)

# --- Nivel 2 ---
# Condición A: PRCS < 0.25 MPa (250000 Pa) AND Pcont > 0.2 MPa (200000 Pa)
cond_A = (PRCS < 250_000.0) & (P_dw > 200_000.0)

# Condición B: CEC > EQ en Drywell (T o P)
cond_B = (T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)

niveles_cond_vpt[cond_A | cond_B] = 2

# --- Nivel 3 ---
niveles_cond_vpt[dur_T422_dw >= 1800.0] = 3  # 30 min (1800 s) continuos tras T_dw > 422 K

# --- Nivel 5 ---
niveles_cond_vpt[T_dw > 1530.0] = 5  # nivel 4 omitido

# Aplicar histéresis de 1800 s para bajar
VesselPressureTransmitters_Level = aplicar_histeresis(tiempo_s, niveles_cond_vpt, 1800.0)

# Guardar serie de niveles
guardar_nivel_instrumento("VesselPressureTransmitters", VesselPressureTransmitters_Level)

# Referencias Drywell
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell

# =========================
# Feedwater Flow Transmitters
# Niveles: 1(base), 2: CEC>EQ, 3: 1.5*EQ, 5: 2*EQ (4 omitido)
# =========================
niveles_cond_fwft = np.ones_like(tiempo_s, dtype=int)
niveles_cond_fwft[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2
niveles_cond_fwft[(T_dw > 1.5*EQ_TEMP) | (P_dw > 1.5*EQ_PRESSURE)] = 3
niveles_cond_fwft[(T_dw > 2.0*EQ_TEMP) | (P_dw > 2.0*EQ_PRESSURE)] = 5
FeedwaterFlowTransmitters_Level = aplicar_histeresis(tiempo_s, niveles_cond_fwft, 1800.0)
guardar_nivel_instrumento("FeedwaterFlowTransmitters", FeedwaterFlowTransmitters_Level)

# =========================
# Drywell Thermocouples
# Niveles: 1 (273–422 K base), 2: CEC>EQ, 3: T>673 K, 5: T>1530 K (4 omitido)
# =========================
niveles_cond_dwtc = np.ones_like(tiempo_s, dtype=int)
# (si quieres forzar estrictamente 273–422K para nivel 1, dilo y lo ajusto)
niveles_cond_dwtc[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2
niveles_cond_dwtc[T_dw > 673.0] = 3
niveles_cond_dwtc[T_dw > 1530.0] = 5
DrywellThermocouples_Level = aplicar_histeresis(tiempo_s, niveles_cond_dwtc, 1800.0)
guardar_nivel_instrumento("DrywellThermocouples", DrywellThermocouples_Level)

# =========================
# Radiation Detectors in Drywell
# Niveles: 1(base), 2: CEC>EQ, 3: 1.5*EQ, 5: 2*EQ (4 omitido)
# =========================
niveles_cond_rddw = np.ones_like(tiempo_s, dtype=int)
niveles_cond_rddw[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2
niveles_cond_rddw[(T_dw > 1.5*EQ_TEMP) | (P_dw > 1.5*EQ_PRESSURE)] = 3
niveles_cond_rddw[(T_dw > 2.0*EQ_TEMP) | (P_dw > 2.0*EQ_PRESSURE)] = 5
RadiationDetectorsDrywell_Level = aplicar_histeresis(tiempo_s, niveles_cond_rddw, 1800.0)
guardar_nivel_instrumento("RadiationDetectorsDrywell", RadiationDetectorsDrywell_Level)

# =========================
# Control Rods Position
# Niveles: 1(base), 2: CEC>EQ, 5: Vessel Failure (3 y 4 omitidos)
# =========================
niveles_cond_crp = np.ones_like(tiempo_s, dtype=int)
niveles_cond_crp[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2
niveles_cond_crp[VESSEL_FAILURE] = 5
ControlRodsPosition_Level = aplicar_histeresis(tiempo_s, niveles_cond_crp, 1800.0)
guardar_nivel_instrumento("ControlRodsPosition", ControlRodsPosition_Level)

# =========================
# INSTRUMENTO: Recirculation Pressure Transmitters (P106 / T106)
# =========================
P106 = Presion_BombaRecir        # Pa
T106 = TempVapor_BombaRecir      # K

# Umbrales en Pa/K (conversión MPa -> Pa incluida)
P2_thr = 8_000_000.0     # 8 MPa
T2_thr = 573.0           # 573 K
P3_thr = 12_000_000.0    # 12 MPa
T3_thr = 1100.0          # 1100 K
P4_thr = 17_200_000.0    # 17.2 MPa
T4_thr = 1530.0          # 1530 K

# Niveles por condiciones instantáneas (1..5)
niveles_cond_rpt = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto (base)

# Nivel 2
niveles_cond_rpt[(P106 > P2_thr) | (T106 > T2_thr)] = 2

# Nivel 3
niveles_cond_rpt[(P106 > P3_thr) | (T106 > T3_thr)] = 3

# Nivel 4
niveles_cond_rpt[(P106 > P4_thr) | (T106 > T4_thr)] = 4

# Nivel 5 (Vessel Failure)
niveles_cond_rpt[VESSEL_FAILURE] = 5

# Aplicar histéresis de 1800 s para bajar
RecirculationPressureTransmitters_Level = aplicar_histeresis(tiempo_s, niveles_cond_rpt, 10000.0)

# Guardar serie de niveles
guardar_nivel_instrumento("RecirculationPressureTransmitters", RecirculationPressureTransmitters_Level)

# =========================
# INSTRUMENTO: Recirculation Flow Transmitters (P106 / T106)
# =========================
P106 = Presion_BombaRecir   # Pa
T106 = TempVapor_BombaRecir # K

# Umbrales (MPa -> Pa)
P2_thr = 8_000_000.0     # 8 MPa
T2_thr = 573.0           # 573 K
P3_thr = 12_000_000.0    # 12 MPa
T3_thr = 1100.0          # 1100 K
P4_thr = 17_200_000.0    # 17.2 MPa
T4_thr = 1530.0          # 1530 K

# Niveles por condiciones instantáneas (1..5)
niveles_cond_rft = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto

# Nivel 2
niveles_cond_rft[(P106 > P2_thr) | (T106 > T2_thr)] = 2

# Nivel 3
niveles_cond_rft[(P106 > P3_thr) | (T106 > T3_thr)] = 3

# Nivel 4
niveles_cond_rft[(P106 > P4_thr) | (T106 > T4_thr)] = 4

# Nivel 5 (Vessel Failure)
niveles_cond_rft[VESSEL_FAILURE] = 5

# Histéresis: 1800 s para bajar de nivel
RecirculationFlowTransmitters_Level = aplicar_histeresis(tiempo_s, niveles_cond_rft, 10000.0)

# Guardar serie de niveles
guardar_nivel_instrumento("RecirculationFlowTransmitters", RecirculationFlowTransmitters_Level)

# =========================
# INSTRUMENTO: Average Reactor Power Monitors (Drywell)
# Niveles: 1(base), 2: CEC>EQ, 5: Vessel Failure (3 y 4 omitidos)
# =========================
T_dw = TempVapor_Drywell
P_dw = Presion_Drywell

niveles_cond_arpm = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto

# Nivel 2: CEC>EQ (OR en T o P del Drywell)
niveles_cond_arpm[(T_dw > EQ_TEMP) | (P_dw > EQ_PRESSURE)] = 2

# Nivel 5: Vessel Failure
niveles_cond_arpm[VESSEL_FAILURE] = 5

# Histéresis: 1800 s para bajar de nivel
AverageReactorPowerMonitors_Level = aplicar_histeresis(tiempo_s, niveles_cond_arpm, 1800.0)

# Guardar serie de niveles
guardar_nivel_instrumento("AverageReactorPowerMonitors", AverageReactorPowerMonitors_Level)

# =========================
# INSTRUMENTO: Vessel Thermocouples (Upper Head)
# Niveles (1–5):
# 1: 273–645 K (base)
# 2: 645 K < T < 873 K
# 3: 923 K < T < 1173 K
# 4: 1173 K < T < 1530 K
# 5: T > 1570 K
# (Quedan huecos 873–923 K y 1530–1570 K tal como indican los rangos)
# =========================
T_uh = TempVapor_CabezaSuperior
P_uh = Presion_CabezaSuperior  # No interviene en estos criterios, pero lo dejamos referenciado

niveles_cond_vtuh = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto

# Nivel 2: 645 K < T < 873 K
niveles_cond_vtuh[(T_uh > 645.0) & (T_uh < 873.0)] = 2

# Nivel 3: 923 K < T < 1173 K
niveles_cond_vtuh[(T_uh > 923.0) & (T_uh < 1173.0)] = 3

# Nivel 4: 1173 K < T < 1530 K
niveles_cond_vtuh[(T_uh > 1173.0) & (T_uh < 1530.0)] = 4

# Nivel 5: T > 1570 K
niveles_cond_vtuh[T_uh > 1570.0] = 5

# Histéresis: 1800 s para bajar de nivel
VesselThermocouplesUpperHead_Level = aplicar_histeresis(tiempo_s, niveles_cond_vtuh, 1800.0)

# Guardar serie de niveles
guardar_nivel_instrumento("VesselThermocouplesUpperHead", VesselThermocouplesUpperHead_Level)

# =========================
# INSTRUMENTO: Vessel Thermocouples (Lower Head)
# Niveles (1–5):
# 1: 273–645 K (base)
# 2: 645 K < T < 873 K
# 3: 923 K < T < 1173 K
# 4: 1173 K < T < 1530 K
# 5: T > 1570 K
# (Quedan huecos 873–923 K y 1530–1570 K, igual que el de Upper Head)
# =========================
T_lh = TempVapor_CabezaInferior
P_lh = Presion_CabezaInferior  # no usado en estos criterios

niveles_cond_vtlh = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto

# Nivel 2: 645 K < T < 873 K
niveles_cond_vtlh[(T_lh > 645.0) & (T_lh < 873.0)] = 2

# Nivel 3: 923 K < T < 1173 K
niveles_cond_vtlh[(T_lh > 923.0) & (T_lh < 1173.0)] = 3

# Nivel 4: 1173 K < T < 1530 K
niveles_cond_vtlh[(T_lh > 1173.0) & (T_lh < 1530.0)] = 4

# Nivel 5: Vessel Failure
niveles_cond_vtlh[VESSEL_FAILURE] = 5

# Histéresis: 1800 s para bajar de nivel
VesselThermocouplesLowerHead_Level = aplicar_histeresis(tiempo_s, niveles_cond_vtlh, 20000.0)

# Guardar serie de niveles
guardar_nivel_instrumento("VesselThermocouplesLowerHead", VesselThermocouplesLowerHead_Level)

# =========================
# INSTRUMENTO: FW Thermocouples (usando Cabeza Inferior como proxy de Tcore)
# Niveles (1–5):
# 1: 273–645 K (base)
# 2: 645 K < Tcore < 873 K
# 3: — (omitido)
# 4: 873 K < Tcore < 1530 K
# 5: Tcore > 1570 K
# =========================
Tcore = TempVapor_CabezaInferior  # proxy Tcore con Cabeza Inferior

niveles_cond_fwtc = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto

# Nivel 2
niveles_cond_fwtc[(Tcore > 645.0) & (Tcore < 873.0)] = 2

# Nivel 4
niveles_cond_fwtc[(Tcore > 873.0) & (Tcore < 1530.0)] = 4

# Nivel 5
niveles_cond_fwtc[Tcore > 1570.0] = 5

# Histéresis: 1800 s para bajar de nivel
FWThermocouples_Level = aplicar_histeresis(tiempo_s, niveles_cond_fwtc, 25000.0)

# Guardar serie de niveles
guardar_nivel_instrumento("FWThermocouples", FWThermocouples_Level)

# =========================
# INSTRUMENTO: Suction Pipes Thermocouples (T106)
# Niveles (1–5):
# 1: 273–645 K (base)
# 2: 645 K < T < 873 K
# 3: 923 K < T < 1173 K
# 4: 1173 K < T < 1530 K
# 5: T > 1570 K
# (Quedan huecos 873–923 K y 1530–1570 K, según tu tabla)
# =========================
T106 = TempVapor_BombaRecir  # K

niveles_cond_sptc = np.ones_like(tiempo_s, dtype=int)  # Nivel 1 por defecto

# Nivel 2
niveles_cond_sptc[(T106 > 645.0) & (T106 < 873.0)] = 2
# Nivel 3
niveles_cond_sptc[(T106 > 923.0) & (T106 < 1173.0)] = 3
# Nivel 4
niveles_cond_sptc[(T106 > 1173.0) & (T106 < 1530.0)] = 4
# Nivel 5
niveles_cond_sptc[T106 > 1570.0] = 5

# Histéresis: 1800 s para bajar de nivel
SuctionPipesThermocouples_Level = aplicar_histeresis(tiempo_s, niveles_cond_sptc, 100000.0)

# Guardar serie de niveles
guardar_nivel_instrumento("SuctionPipesThermocouples", SuctionPipesThermocouples_Level)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def exportar_niveles_y_graficas(niveles_instrumentos: dict, tiempo_s: np.ndarray,
                                carpeta_csv: str = "csv_niveles",
                                carpeta_figs: str = "graficas_niveles"):
    """
    Para cada instrumento en `niveles_instrumentos`:
      - Guarda CSV con columnas: tiempo_s, nivel
      - Genera PNG con step-plot del nivel (1–5) vs tiempo
    """
    os.makedirs(carpeta_csv, exist_ok=True)
    os.makedirs(carpeta_figs, exist_ok=True)

    for nombre, niveles in niveles_instrumentos.items():
        if niveles.shape != tiempo_s.shape:
            print(f"[ADVERTENCIA] Longitud diferente en '{nombre}': "
                  f"tiempo={tiempo_s.shape}, niveles={niveles.shape}. Se omite.")
            continue

        # Nombre de archivo seguro
        safe = "".join(c if c.isalnum() or c in ("_", "-") else "_" for c in nombre)

        # ----- CSV -----
        df = pd.DataFrame({"tiempo_s": tiempo_s, "nivel": niveles.astype(int)})
        ruta_csv = os.path.join(carpeta_csv, f"{safe}_niveles.csv")
        df.to_csv(ruta_csv, index=False)

        # ----- Gráfica -----
        plt.figure()
        plt.step(tiempo_s, niveles, where="post")
        plt.xlabel("Tiempo [s]")
        plt.ylabel("Nivel de daño (1–5)")
        plt.title(f"{nombre} – Nivel (1–5)")
        plt.yticks([1, 2, 3, 4, 5])
        plt.ylim(0.8, 5.2)  # para ver bien los extremos
        plt.grid(True)
        plt.tight_layout()
        ruta_png = os.path.join(carpeta_figs, f"{safe}_niveles.png")
        plt.savefig(ruta_png, dpi=200)
        plt.close()

        print(f"Guardado CSV: {ruta_csv}")
        print(f"Guardada gráfica: {ruta_png}")

# Ejecuta la exportación
exportar_niveles_y_graficas(niveles_instrumentos, tiempo_s)



Guardado CSV: csv_niveles\SuppressionPoolThermocouples_niveles.csv
Guardada gráfica: graficas_niveles\SuppressionPoolThermocouples_niveles.png
Guardado CSV: csv_niveles\SRVPositionIndicators_niveles.csv
Guardada gráfica: graficas_niveles\SRVPositionIndicators_niveles.png
Guardado CSV: csv_niveles\RadiationDetectorsContainment_niveles.csv
Guardada gráfica: graficas_niveles\RadiationDetectorsContainment_niveles.png
Guardado CSV: csv_niveles\BoronInjectionPressureTransmitters_niveles.csv
Guardada gráfica: graficas_niveles\BoronInjectionPressureTransmitters_niveles.png
Guardado CSV: csv_niveles\BoronTankLevelTransmitter_niveles.csv
Guardada gráfica: graficas_niveles\BoronTankLevelTransmitter_niveles.png
Guardado CSV: csv_niveles\DrywellPressureTransmitters_niveles.csv
Guardada gráfica: graficas_niveles\DrywellPressureTransmitters_niveles.png
Guardado CSV: csv_niveles\ContainmentPressureTransmitters_niveles.csv
Guardada gráfica: graficas_niveles\ContainmentPressureTransmitters_niveles.png
G

In [16]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ============================================================
# CONFIGURACIÓN
# ============================================================
carpeta = Path("csv_niveles")

state_order = {'Good':0, 'Careful':1, 'Warning':2, 'Blind':3}

# ============================================================
# MAPEO CSV POR INSTRUMENTO
# ============================================================
instrument_to_csv = {
    "Average Reactor Power Monitors": "AverageReactorPowerMonitors",
    "Boron Injection Pressure Transmitters": "BoronInjectionPressureTransmitters",
    "Boron Tank Level Transmitter": "BoronTankLevelTransmitter",
    "Containment Pressure Transmitters": "ContainmentPressureTransmitters",
    "Containment Thermocouples": "ContainmentThermocouples",
    "Control Rods Position": "ControlRodsPosition",
    "Drywell Pressure Transmitters": "DrywellPressureTransmitters",
    "Drywell Thermocouples": "DrywellThermocouples",
    "Feedwater Flow Transmitters": "FeedwaterFlowTransmitters",
    "Feedwater Pressure Transmitters": "FeedwaterPressureTransmitters",
    "FW Thermocouples": "FWThermocouples",
    "Radiation Detectors in Containment": "RadiationDetectorsContainment",
    "Radiation Detectors in Drywell": "RadiationDetectorsDrywell",
    "Recirculation Flow Transmitters": "RecirculationFlowTransmitters",
    "Recirculation Pressure Transmitters": "RecirculationPressureTransmitters",
    "SRV Position Indicators": "SRVPositionIndicators",
    "Suction Pipes Thermocouples": "SuctionPipesThermocouples",
    "Supression Pool Thermocouples": "SuppressionPoolThermocouples",  # base 'pp'
    "Suppresion Pool Level Transmitters": "SuppressionPoolLevelTransmitters",
    "Vessel Level (Fuel Range)": "VesselLevelFuelRange",
    "Vessel Level (Wide Range)": "VesselLevelWideRange",
    "Vessel Pressure Transmitters": "VesselPressureTransmitters",
    "Vessel Thermocouples (Lower Head)": "VesselThermocouplesLowerHead",
    "Vessel Thermocouples (Upper Head)": "VesselThermocouplesUpperHead",
    "HPCS Flow Transmitter": "HPCSFlowTransmitter",
    "Containment Hydrogen Concentration Analyzer": "ContainmentHydrogenAnalyzer",
    "Drywell Hydrogen Concentration Analyzer": "DrywellHydrogenAnalyzer",
    # Opcionales si existen CSV:
    "Containment Pressure Transmitter (Auxiliary Building)": "ContainmentPressureTransmitterAux",
    "Drywell Pressure Transmitter (Auxiliary Building)": "DrywellPressureTransmitterAux",
}

# Alias (por si aparecen variantes de escritura)
name_aliases = {
    "Suppression Pool Thermocouples": "Supression Pool Thermocouples",
    "Suppression Pool Level Transmitters": "Suppresion Pool Level Transmitters",
}


# ============================================================
# UTILIDADES
# ============================================================
def _canon(s: str) -> str:
    return "".join(ch.lower() for ch in s if ch.isalnum())

def resolve_instrument_name(name: str, available: dict) -> str | None:
    name = name_aliases.get(name, name)
    if name in available:
        return name
    idx = { _canon(k): k for k in available.keys() }
    return idx.get(_canon(name))

def leer_csv_flexible(path: Path) -> pd.DataFrame:
    """Lee CSV con separador auto (, ; \t) y devuelve DF con columnas 'tiempo_s','nivel'."""
    seps = [",",";","\t","|"]
    last_err = None
    for sep in seps:
        try:
            df = pd.read_csv(path, sep=sep)
            if df.shape[1] < 1:
                continue
            # limpiar columnas unnamed
            df = df.loc[:, ~df.columns.astype(str).str.startswith("Unnamed")]
            cols = [c.lower().strip() for c in df.columns.astype(str)]

            # Caso NUEVO (ancho): columnas tiempo_s, nivel
            if "tiempo_s" in cols and "nivel" in cols:
                tcol = df.columns[cols.index("tiempo_s")]
                ncol = df.columns[cols.index("nivel")]
                out = df[[tcol,ncol]].rename(columns={tcol:"tiempo_s", ncol:"nivel"}).copy()
                out["tiempo_s"] = pd.to_numeric(out["tiempo_s"], errors="coerce")
                out["nivel"]    = pd.to_numeric(out["nivel"], errors="coerce")
                out = out.dropna(subset=["tiempo_s","nivel"])
                out["nivel"] = out["nivel"].round().clip(1,5).astype(int)
                out = out.sort_values("tiempo_s").reset_index(drop=True)
                return out

            # Caso ANTIGUO (largo): Intervalo / Nivel de Daño
            if "intervalo" in cols and "nivel de daño" in cols:
                icol = df.columns[cols.index("intervalo")]
                lcol = df.columns[cols.index("nivel de daño")]
                tmp = df[[icol,lcol]].rename(columns={icol:"Intervalo", lcol:"Nivel de Daño"}).copy()
                tmp["Intervalo"] = tmp["Intervalo"].astype(str)
                tmp["Nivel de Daño"] = pd.to_numeric(tmp["Nivel de Daño"], errors="coerce").fillna(1).astype(int).clip(1,5)
                # inventamos tiempo_s como 0,1,2,... (pasos)
                out = pd.DataFrame({
                    "tiempo_s": np.arange(len(tmp), dtype=float),
                    "nivel": tmp["Nivel de Daño"].to_numpy(int)
                })
                return out

            # Si tiene dos columnas cualesquiera, intentamos mapear a tiempo,nivel por posición
            if df.shape[1] >= 2:
                out = df.iloc[:, :2].copy()
                out.columns = ["tiempo_s","nivel"]
                out["tiempo_s"] = pd.to_numeric(out["tiempo_s"], errors="coerce")
                out["nivel"]    = pd.to_numeric(out["nivel"], errors="coerce")
                out = out.dropna(subset=["tiempo_s","nivel"])
                out["nivel"] = out["nivel"].round().clip(1,5).astype(int)
                out = out.sort_values("tiempo_s").reset_index(drop=True)
                return out

        except Exception as e:
            last_err = e
            continue
    # Si no pudimos leer, relanza último error
    raise last_err if last_err else RuntimeError(f"No se pudo leer {path}")

# ====== NUEVO: utilidades para construir y guardar CSV ======
def _levels_dataframe(instrs):
    """
    Devuelve (t_ref, df) donde df contiene 'tiempo_s' y,
    por cada instrumento, una columna con su nivel remuestreado (1..5).
    """
    t_ref = _reference_time(instrs)
    data = {"tiempo_s": t_ref}
    for i_name in instrs:
        resolved = resolve_instrument_name(i_name, damage_by_instrument) or i_name
        df = damage_by_instrument.get(resolved)
        y = _resample_to_reference(df, t_ref)
        data[resolved] = y.astype(int)
    out = pd.DataFrame(data)
    return t_ref, out

def _availability_dataframe(instrs):
    """
    Calcula availability y devuelve (t_ref, df) con columnas:
    'tiempo_s', 'state', 'state_code' (0..3).
    """
    t_ref = _reference_time(instrs)
    levels = []
    for i_name in instrs:
        resolved = resolve_instrument_name(i_name, damage_by_instrument) or i_name
        df = damage_by_instrument.get(resolved)
        y = _resample_to_reference(df, t_ref)
        levels.append(y.astype(int))

    states, codes = [], []
    for i in range(len(t_ref)):
        lvls_t = [int(arr[i]) for arr in levels]
        st = determine_state_from_levels(lvls_t)
        states.append(st)
        codes.append(state_order[st])

    df = pd.DataFrame({
        "tiempo_s": t_ref,
        "state": states,
        "state_code": np.array(codes, dtype=int)
    })
    return t_ref, df
# ============================================================


# ============================================================
# CARGA TODOS LOS INSTRUMENTOS + TIEMPO DE REFERENCIA
# ============================================================
raw_data = {}
ref_time = None

for visible_name, base in instrument_to_csv.items():
    ruta = carpeta / f"{base}_niveles.csv"
    if ruta.exists():
        df = leer_csv_flexible(ruta)
        raw_data[visible_name] = df
        if ref_time is None and not df.empty:
            ref_time = df["tiempo_s"].to_numpy()
    else:
        # se rellena luego con fallback
        pass

# Si no hay ningún CSV real, inventamos ref_time
if ref_time is None:
    ref_time = np.arange(100, dtype=float)  # 100 puntos por defecto

# Completar faltantes con nivel=1 y mismo eje temporal
for visible_name, base in instrument_to_csv.items():
    if visible_name not in raw_data:
        print(f"⚠ No encontrado: {base}_niveles.csv → se asigna nivel=1 con la malla temporal de referencia")
        raw_data[visible_name] = pd.DataFrame({
            "tiempo_s": ref_time,
            "nivel": np.ones_like(ref_time, dtype=int)
        })

# ============================================================
# PREPARACIÓN DE ESTRUCTURAS
# ============================================================
damage_by_instrument = {name: df for name, df in raw_data.items()}

# ============================================================
# LÓGICA AVAILABILITY (tu regla)
# ============================================================
def determine_state_from_levels(lvls: list[int]) -> str:
    if all(l == 1 for l in lvls):
        return 'Good'
    if any(l == 2 for l in lvls) and all(l <= 2 for l in lvls):
        return 'Careful'
    if all(l == 5 for l in lvls):
        return 'Blind'
    if any(l in (3,4) for l in lvls):
        return 'Warning'
    return 'Warning'

# ============================================================
# PLOTS
# ============================================================
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

def _reference_time(instruments):
    """Devuelve el vector tiempo_s de referencia (el del primer instrumento con datos)."""
    for instr in instruments:
        resolved = resolve_instrument_name(instr, damage_by_instrument) or instr
        df = damage_by_instrument.get(resolved)
        if df is not None and not df.empty:
            return df["tiempo_s"].to_numpy()
    return ref_time

def _resample_to_reference(df: pd.DataFrame, t_ref: np.ndarray) -> np.ndarray:
    """
    Ajusta la serie (tiempo_s, nivel) al eje temporal t_ref:
    - si longitudes coinciden y tiempos iguales ≈: devuelve niveles tal cual
    - si difiere, interpola 'nearest' por escalón (usamos pandas merge_asof).
    """
    if df is None or df.empty:
        return np.ones_like(t_ref, dtype=int)
    src_t = df["tiempo_s"].to_numpy()
    src_y = df["nivel"].to_numpy(int)

    if len(src_t) == len(t_ref) and np.allclose(src_t, t_ref, rtol=0, atol=1e-9):
        return src_y

    # merge_asof para asignar al tiempo de referencia el nivel más cercano (look-up tipo nearest)
    a = pd.DataFrame({"t": t_ref})
    b = pd.DataFrame({"t": src_t, "y": src_y}).sort_values("t")
    out = pd.merge_asof(a.sort_values("t"), b, on="t", direction="nearest")
    y = out["y"].fillna(1).round().clip(1,5).astype(int).to_numpy()
    return y

def plot_damage(instruments, save_path, title):
    t_ref = _reference_time(instruments)
    fig, ax = plt.subplots(figsize=(16, 8), dpi=100)
    fig.patch.set_facecolor('#001F3F')
    ax.set_facecolor('#001F3F')

    markers = ['o','s','D','^','v','P','X','*','<','>']
    for idx, instr in enumerate(instruments):
        resolved = resolve_instrument_name(instr, damage_by_instrument) or instr
        df = damage_by_instrument.get(resolved)
        y = _resample_to_reference(df, t_ref)
        jitter = (idx - len(instruments)/2) * 0.05
        ax.plot(
            t_ref, y + jitter,
            marker=markers[idx % len(markers)],
            markersize=6,
            linewidth=1.8,
            label=resolved,
            markevery=max(1, len(t_ref)//20)
        )

    ax.set_xlabel("Time (s)", color='white', fontsize=20)
    ax.set_ylabel("Damage Level", color='white', fontsize=20)
    ax.set_yticks([1,2,3,4,5])
    ax.set_yticklabels([1,2,3,4,5], color='white', fontsize=18)
    ax.tick_params(colors='white', labelsize=16)
    for spine in ax.spines.values():
        spine.set_edgecolor('white')
    ax.grid(color='white', linestyle='--', alpha=0.2, linewidth=1)
    ax.set_title(title, color='white', fontsize=28, pad=24)

    # leyenda abajo
    leg_cols = len(instruments) if len(instruments) <= 2 else (len(instruments)+1)//2
    leg = ax.legend(
        loc='upper center', bbox_to_anchor=(0.5, -0.12),
        ncol=max(1, leg_cols), frameon=False, fontsize=14
    )
    for txt in leg.get_texts():
        txt.set_color('white')

    fig.subplots_adjust(bottom=0.25)
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=300)
    plt.close(fig)

def plot_availability(measure, t_ref, states, save_path):
    import numpy as np
    values = np.array([state_order[s] for s in states], dtype=int)

    fig, ax = plt.subplots(figsize=(16, 8), dpi=100)
    fig.patch.set_facecolor('#001F3F')
    ax.set_facecolor('#001F3F')

    # bandas por estado (Good/Careful/Warning/Blind)
    bands = {0:'#2E7D32', 1:'#F9A825', 2:'#EF6C00', 3:'#B71C1C'}
    for lvl, col in bands.items():
        ax.axhspan(lvl, lvl+1, color=col, alpha=0.25)

    # línea escalonada (mismo estilo que daño)
    ax.plot(t_ref, values, drawstyle='steps-post', marker='o',
            markersize=6, linewidth=1.8)

    # === Igualamos estilo de ejes al de "daño" ===
    ax.set_xlabel("Time (s)", color='white', fontsize=20)
    ax.set_ylabel("", color='white', fontsize=20)  # sin etiqueta textual
    ax.set_yticks([0,1,2,3])
    ax.set_yticklabels(list(state_order.keys()), color='white', fontsize=18)

    # ¡NO fijamos los xticks! (dejamos el auto-locator como en daño)
    ax.tick_params(colors='white', labelsize=16)
    for spine in ax.spines.values():
        spine.set_edgecolor('white')
    ax.grid(color='white', linestyle='--', alpha=0.2, linewidth=1)
    ax.set_title(f"{measure} Availability", color='white', fontsize=28, pad=24)

    fig.subplots_adjust(bottom=0.25)
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=300)
    plt.close(fig)


# ============================================================
# GENERACIÓN DE SALIDAS
# ============================================================
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

def generate_graphs(samg_name, samg_dict, actions_map):
    create_directory(samg_name)
    for subguide, codes in samg_dict.items():
        path_sub = os.path.join(samg_name, subguide)
        create_directory(path_sub)
        for code in codes:
            if code not in actions_map:
                continue
            path_code = os.path.join(path_sub, code)
            create_directory(path_code)
            for measure, instrs in actions_map[code].items():
                base = measure.replace(' ', '_').replace('/', '_')
                # --- nuevo: CSV de niveles por instrumento (remuestreados) ---
                _, df_levels = _levels_dataframe(instrs)
                df_levels.to_csv(os.path.join(path_code, base + ".csv"),
                                 index=False, encoding="utf-8")
                # --- gráfico como antes ---
                plot_damage(instrs, os.path.join(path_code, base + ".png"), measure)


def generate_availability(samg_name, samg_dict, actions_map):
    base_dir = f"Availability_{samg_name}"
    create_directory(base_dir)
    for subguide, codes in samg_dict.items():
        path_sub = os.path.join(base_dir, subguide)
        create_directory(path_sub)
        for code in codes:
            if code not in actions_map:
                continue
            path_code = os.path.join(path_sub, code)
            create_directory(path_code)
            for measure, instrs in actions_map[code].items():
                safe = measure.replace(' ', '_').replace('/', '_')
                # --- nuevo: CSV de availability ---
                t_ref, df_av = _availability_dataframe(instrs)
                df_av.to_csv(os.path.join(path_code, safe + "_availability.csv"),
                             index=False, encoding="utf-8")
                # --- gráfico como antes ---
                states = df_av["state"].tolist()
                plot_availability(measure, t_ref, states,
                                  os.path.join(path_code, safe + "_availability.png"))


# ============================================================
# EJECUCIÓN
# ============================================================
generate_graphs("SAMG1_PyT", SAMG1, actions_short)
generate_graphs("SAMG2_PyT", SAMG2, actions_short)
generate_availability("SAMG1_PyT", SAMG1, actions_short)
generate_availability("SAMG2_PyT", SAMG2, actions_short)


⚠ No encontrado: FeedwaterPressureTransmitters_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: SuppressionPoolLevelTransmitters_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: HPCSFlowTransmitter_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: ContainmentHydrogenAnalyzer_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: DrywellHydrogenAnalyzer_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: ContainmentPressureTransmitterAux_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: DrywellPressureTransmitterAux_niveles.csv → se asigna nivel=1 con la malla temporal de referencia


In [17]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ===========================
# CONFIGURATION
# ===========================

VARIABLES = [
    "TEMPERATURE_DRYWELL",
    "PRESSURE_DRYWELL",
    "RADIATION_CONTAINMENT",
    "HYDROGEN_CONTAINMENT",
    "LEVEL_BORON_TANK",
    "INJECTED_FLOW_RPV",
    "LEVEL_RPV",
    "PRESSURE_RPV",
    "TEMPERATURE_RPV",
    "PRESSURE_CONTAINMENT",
    "LEVEL_CONTAINMENT",
    "PRESSURE_SP",
    "TEMPERATURE_SP",
    "SRV_POSITION",
    "REACTOR_POWER",
    "TEMPERATURE_CONTAINMENT"
]

OUT_DIR = "GLOBAL_EVALUATION"
os.makedirs(OUT_DIR, exist_ok=True)

T_MAX = 86400

MARKERS = ["o", "s", "^", "D", "v", "P", "X", "<", ">", "h"]

STATE_MAP = {"Good": 0, "Careful": 1, "Warning": 2, "Blind": 3}
ZONE_COLORS = {0: "#006400", 1: "#CCCC00", 2: "#FF8C00", 3: "#8B0000"}

# ===========================
# UTILITIES
# ===========================

def interval_to_seconds(interval_str: str):
    norm = str(interval_str).strip().lower().replace(" ", "")
    if norm in ("12h-1d", "12h-1dia", "12h-1día", "12h_a_1d"):
        return (12*3600, 24*3600)
    if norm in ("0-1d", "0-1dia", "0-1día"):
        return (12*3600, 24*3600)
    if norm in ("0-1h", "0h-1h"):
        return (0, 1*3600)
    if norm in ("1-12h", "1h-12h"):
        return (1*3600, 12*3600)
    if "-" in norm:
        a, b = norm.split("-")
        def to_s(x):
            if x.endswith("h"):
                return int(float(x[:-1]) * 3600)
            if x.endswith("d"):
                return int(float(x[:-1]) * 86400)
            return int(float(x))
        return (to_s(a), to_s(b))
    raise ValueError(f"No puedo interpretar el intervalo: {interval_str!r}")

def load_interval_table(csv1_path: str) -> pd.DataFrame:
    df1 = pd.read_csv(csv1_path)
    col_interval = None
    for c in df1.columns:
        if str(c).strip().lower() == "interval":
            col_interval = c
            break
    if col_interval is None:
        raise ValueError(f"{csv1_path}: no encuentro columna 'Interval'.")
    inst_cols = [c for c in df1.columns if c != col_interval]
    rows = []
    for _, r in df1.iterrows():
        start, end = interval_to_seconds(r[col_interval])
        for col in inst_cols:
            val = int(r[col])
            rows.append({"start": start, "end": end, "Instrument": col, "Damage": val})
    return pd.DataFrame(rows).sort_values(["Instrument", "start"]).reset_index(drop=True)

def load_timeseries_table(csv2_path: str) -> pd.DataFrame:
    df2 = pd.read_csv(csv2_path)
    time_col = None
    for c in df2.columns:
        if str(c).strip().lower() in ("tiempo_s", "time_s", "t", "tiempo"):
            time_col = c
            break
    if time_col is None:
        raise ValueError(f"{csv2_path}: no encuentro columna de tiempo en segundos.")
    if time_col != "tiempo_s":
        df2 = df2.rename(columns={time_col: "tiempo_s"})
    return df2

def df1_value_at(df1_intervals: pd.DataFrame, instrument: str, t: float) -> int:
    sub = df1_intervals[(df1_intervals["Instrument"] == instrument) &
                        (df1_intervals["start"] <= t) &
                        (t < df1_intervals["end"])]
    if not sub.empty:
        return int(sub.iloc[0]["Damage"])
    if math.isclose(t, T_MAX, rel_tol=0, abs_tol=1e-9):
        last = df1_intervals[df1_intervals["Instrument"] == instrument].sort_values("end").iloc[-1]
        return int(last["Damage"])
    return np.nan

def df2_value_at(df2: pd.DataFrame, instrument: str, t: float) -> int:
    sub = df2[df2["tiempo_s"] <= t]
    if sub.empty:
        return int(df2.iloc[0][instrument])
    return int(sub.iloc[-1][instrument])

def build_series_max(csv1_path: str, csv2_path: str):
    df1_intervals = load_interval_table(csv1_path)
    df2 = load_timeseries_table(csv2_path)
    insts = sorted(set(df1_intervals["Instrument"].unique()).union(
                   set([c for c in df2.columns if c != "tiempo_s"])))
    T = set([0, T_MAX])
    for _, r in df1_intervals.iterrows():
        T.add(int(r["start"]))
        T.add(int(r["end"]))
    for t in df2["tiempo_s"].values:
        if 0 <= t <= T_MAX:
            T.add(float(t))
    T = sorted(T)
    series = {}
    for inst in insts:
        y_vals = []
        for tt in T:
            v1 = df1_value_at(df1_intervals, inst, tt)
            v2 = df2_value_at(df2, inst, tt) if inst in df2.columns else np.nan
            y_vals.append(int(max(v1, v2)))
        x_comp, y_comp = [T[0]], [y_vals[0]]
        for i in range(1, len(T)):
            if y_vals[i] != y_comp[-1]:
                x_comp.append(T[i])
                y_comp.append(y_vals[i])
        if x_comp[-1] != T_MAX:
            x_comp.append(T_MAX)
            y_comp.append(y_comp[-1])
        series[inst] = (np.array(x_comp), np.array(y_comp))
    return series, insts

def auto_offsets(n: int, base=0.06):  # más separación que antes
    if n == 1:
        return [0.0]
    idx = np.arange(n) - (n-1)/2.0
    return (idx / max(1, (n-1)/2.0)) * base

def plot_variable(name: str, series: dict, insts: list, out_dir=OUT_DIR):
    fig, ax = plt.subplots(figsize=(12, 6))
    fig.patch.set_facecolor("#06243A")
    ax.set_facecolor("#06243A")
    offs = auto_offsets(len(insts), base=0.06)
    markers = {insts[i]: MARKERS[i % len(MARKERS)] for i in range(len(insts))}
    line_kwargs = dict(
        where="post",
        linewidth=2.0,
        markersize=7,
        markeredgewidth=1.8,
        solid_capstyle="butt",
        mfc="none",
        markevery=None
    )
    y_all = []
    for i, inst in enumerate(insts):
        x, y = series[inst]
        y_shift = y + offs[i]
        y_all.extend(list(y_shift))
        ln = ax.step(x, y_shift, marker=markers[inst], label=inst, **line_kwargs)[0]
        ln.set_zorder(3)
    ax.set_title(name.replace("_", " "), fontsize=34, fontweight="bold", color="white", pad=20)
    ax.set_xlabel("Time (s)", fontsize=20, color="white", labelpad=15)
    ax.set_ylabel("Damage Level", fontsize=24, color="white", labelpad=15)
    ax.set_xlim(0, T_MAX)
    ymin = max(0.9, np.nanmin(y_all) - 0.1)
    ymax = min(5.1, np.nanmax(y_all) + 0.1)
    ax.set_ylim(ymin, ymax)
    ax.tick_params(axis="both", colors="white", labelsize=14)
    ax.grid(True, linestyle="--", alpha=0.35)
    for spine in ax.spines.values():
        spine.set_color("white")
        spine.set_alpha(0.6)
    plt.subplots_adjust(bottom=0.24)
    leg_cols = min(4, max(2, int(math.ceil(len(insts)/2))))
    legend = ax.legend(
        loc="upper center",
        bbox_to_anchor=(0.5, -0.18),
        ncol=leg_cols,
        frameon=False,
        fontsize=14,
        handlelength=2.6,
        handletextpad=0.8,
        columnspacing=1.8,
    )
    for text in legend.get_texts():
        text.set_color("white")
    plt.tight_layout()
    out_path = os.path.join(out_dir, f"{name}_COMBINED.png")
    plt.savefig(out_path, dpi=220, bbox_inches="tight")
    plt.close(fig)
    print(f"✓ Guardado: {out_path}")

# ===========================
# AVAILABILITY
# ===========================

def classify_state_from_levels(levels):
    if not levels:
        return "Blind"
    lvls = [int(x) for x in levels if x is not None]
    if all(l == 1 for l in lvls):
        return "Good"
    if all(l == 5 for l in lvls):
        return "Blind"
    if any(l in (3, 4) for l in lvls):
        return "Warning"
    if any(l == 2 for l in lvls) and all(l <= 2 for l in lvls):
        return "Careful"
    return "Warning"

def build_availability_from_series(series: dict, insts: list):
    all_times = set()
    for inst in insts:
        x, _ = series[inst]
        all_times.update(x.tolist())
    times = sorted(all_times)
    states, codes = [], []
    for t in times:
        lvls = []
        for inst in insts:
            x, y = series[inst]
            idx = np.searchsorted(x, t, side="right") - 1
            idx = max(0, min(idx, len(y) - 1))
            lvls.append(int(y[idx]))
        st = classify_state_from_levels(lvls)
        states.append(st)
        codes.append(STATE_MAP[st])
    return pd.DataFrame({"time_s": times, "state": states, "state_code": codes})

def plot_availability_timeseries(var_name: str, df_av: pd.DataFrame, save_path: str):
    times = df_av["time_s"].tolist()
    values = df_av["state_code"].tolist()
    fig, ax = plt.subplots(figsize=(16, 9), dpi=120)
    fig.patch.set_facecolor("#06243A")
    ax.set_facecolor("#06243A")
    for lvl, col in ZONE_COLORS.items():
        ax.axhspan(lvl, lvl + 1, color=col, alpha=0.28)
    ax.step(times, values, where="post", marker="o", mfc="none",
            markersize=6, linewidth=1.8, color="#00CED1")
    ax.set_xlim(0, T_MAX)
    ax.set_xlabel("Time (s)", fontsize=20, color="white", labelpad=15)
    ax.set_ylabel("Availability State", fontsize=20, color="white", labelpad=15)
    ax.set_yticks(list(STATE_MAP.values()))
    ax.set_yticklabels(list(STATE_MAP.keys()), color="white", fontsize=16)
    ax.set_title(f"{var_name.replace('_',' ')} Availability", color="white", fontsize=28, pad=40)
    ax.tick_params(colors="white", labelsize=14)
    ax.grid(color="white", linestyle="--", alpha=0.25, linewidth=1)
    for spine in ax.spines.values():
        spine.set_edgecolor("white")
    fig.subplots_adjust(top=0.88, bottom=0.18)
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=220)
    plt.close(fig)
    print(f"✓ Guardado: {save_path}")

# ===========================
# EJECUCIÓN
# ===========================

for var in VARIABLES:
    csv1 = f"{var}_1.csv"
    csv2 = f"{var}_2.csv"
    if not (os.path.exists(csv1) and os.path.exists(csv2)):
        print(f"⚠️  Omito {var}: faltan {csv1} o {csv2}")
        continue
    series, insts = build_series_max(csv1, csv2)
    # Plot damage curves
    plot_variable(var, series, insts, out_dir=OUT_DIR)
    # Availability
    df_av = build_availability_from_series(series, insts)
    csv_out = os.path.join(OUT_DIR, f"{var}_availability.csv")
    df_av.to_csv(csv_out, index=False)
    plot_availability_timeseries(var, df_av, os.path.join(OUT_DIR, f"{var}_availability.png"))

⚠️  Omito TEMPERATURE_DRYWELL: faltan TEMPERATURE_DRYWELL_1.csv o TEMPERATURE_DRYWELL_2.csv
⚠️  Omito PRESSURE_DRYWELL: faltan PRESSURE_DRYWELL_1.csv o PRESSURE_DRYWELL_2.csv
⚠️  Omito RADIATION_CONTAINMENT: faltan RADIATION_CONTAINMENT_1.csv o RADIATION_CONTAINMENT_2.csv
⚠️  Omito HYDROGEN_CONTAINMENT: faltan HYDROGEN_CONTAINMENT_1.csv o HYDROGEN_CONTAINMENT_2.csv
⚠️  Omito LEVEL_BORON_TANK: faltan LEVEL_BORON_TANK_1.csv o LEVEL_BORON_TANK_2.csv
⚠️  Omito INJECTED_FLOW_RPV: faltan INJECTED_FLOW_RPV_1.csv o INJECTED_FLOW_RPV_2.csv
⚠️  Omito LEVEL_RPV: faltan LEVEL_RPV_1.csv o LEVEL_RPV_2.csv
⚠️  Omito PRESSURE_RPV: faltan PRESSURE_RPV_1.csv o PRESSURE_RPV_2.csv
⚠️  Omito TEMPERATURE_RPV: faltan TEMPERATURE_RPV_1.csv o TEMPERATURE_RPV_2.csv
⚠️  Omito PRESSURE_CONTAINMENT: faltan PRESSURE_CONTAINMENT_1.csv o PRESSURE_CONTAINMENT_2.csv
⚠️  Omito LEVEL_CONTAINMENT: faltan LEVEL_CONTAINMENT_1.csv o LEVEL_CONTAINMENT_2.csv
⚠️  Omito PRESSURE_SP: faltan PRESSURE_SP_1.csv o PRESSURE_SP_2.csv
⚠